<a href="https://colab.research.google.com/github/wkdwlgus/ktcloud_genai/blob/main/251021_%EA%B3%A0%EA%B8%89_%ED%94%84%EB%A1%AC%ED%94%84%ED%8A%B8_%EC%97%94%EC%A7%80%EB%8B%88%EC%96%B4%EB%A7%81_%EC%9E%A5%EC%A7%80%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **프롬프트 엔지니어링**

## **프롬프트 엔지니어링 개요**

- **텍스트 표현**에 초점을 맞춘 모델
    - BERT나 BERT 파생 모델
- **텍스트 생성**에 초점을 맞춘 모델
    - **GPT 계열** 모델
    - 사용자의 **프로프트(prompt)에 대한 응답으로 텍스트 생성**
- **프롬프트 엔지니어링**(Prompt Engineering)
    - <mark>**생성된 텍스트의 품질을 향상시키기 위해 프롬프트를 설계하는 방법**

- **목표**:
    - <mark>프롬프트를 세심하게 설계하여 LLM이 원하는 응답을 하도록 유도할 수 있다.
- **기억할 점**
    - 프롬프트 최적화는 반복적인 과정이며 실험이 필요하다.
    - 완벽한 프롬프트는 설계는 없으며 앞으로도 가능하지 않다.


### **프롬프트의 기본 구성 요소**

1. **지시사항/명령어** (Instruction)
    - 프롬프트의 핵심으로, AI에게 무엇을 해야 하는지 명확하게 전달합니다
    - 모호한 표현보다는 구체적이고 직접적인 동사를 사용하는 것이 효과적입니다
    - "해줘", "만들어줘", "분석해줘", "요약해줘" 등의 명확한 동사 사용
2. **입력 데이터** (Input Data)
    - 실제로 처리되어야 할 원본 데이터입니다
    - 구조화된 데이터(JSON, CSV)나 비구조화된 데이터(텍스트, 이미지) 모두 가능합니다
3. **출력 지시어** (Output Format)
    - 응답의 구조를 미리 지정하면 일관성 있는 결과를 얻을 수 있습니다
    - 특히 프로그래밍에서 파싱이 필요한 경우 매우 중요합니다

### **지시 기반 프롬프트**

- 지시 기반 프롬프트란?
    - 특정 질문에 답변하거나 특정 작업을 해결하기 위해 사용하는 프롬프트
    - 작업마다 다른 지시가 필요하지만 출력 품질을 향상시키기 위한 프롬프트 기술에는 공통점이 있다.

- **출력 품질 향상을 위한 기술**
    - **구체성** : 원하는 바를 정확히/구체적으로 기술하세요.
    - **환각 대응책** : LLM에게 답변을 알 때만 생성하라고 요청할 수 있습니다. 답을 알지 못하면 `답을 모른다`라고 출력할하라고 구체적으로 지시한다.
    - **순서** : 프롬프트 시작이나 끝에 지시 사항을 전달한다. 특히, 긴 프롬프트의 경우 중간에 있는 정보가 잊힐 수 있다.

## **생성 모델 사용하기**

In [ ]:
import warnings
import logging

# 1. 일반적인 Python 경고(DeprecationWarning 등) 숨기기
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# 2. Transformers 라이브러리 로그 수준 조절 (Warning 이하는 숨기기)
logging.getLogger("transformers").setLevel(logging.ERROR)

### **필요 라이브러리 설치**

In [ ]:
# 파이썬 버전 확인(3.12)
!python --version

Python 3.12.12


In [ ]:
# CUDA 버전 확인 (12.5)
!nvcc --version | grep cuda_

Build cuda_12.5.r12.5/compiler.34385749_0


- llama-cpp-python
    - https://github.com/abetlen/llama-cpp-python/releases
    - llama_cpp_python-0.3.4-cp311-cp311-linux_x86_64.whl
- transformers
- 사용 모델
    - microsoft/Phi-3-mini-4k-instruct


In [ ]:
%%capture
# 사용하는 파이썬과 CUDA 버전에 맞는 llama-cpp-python 패키지를 설치하세요.
# !pip install https://github.com/abetlen/llama-cpp-python/releases/download/v0.3.4-cu124/llama_cpp_python-0.3.4-cp311-cp311-linux_x86_64.whl
!pip install https://github.com/abetlen/llama-cpp-python/releases/download/v0.3.16-cu124/llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl

In [ ]:
# Phi-3 모델과 호환성 때문에 transformers 4.48.3 버전을 사용합니다.
!pip install transformers==4.48.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1


In [ ]:
!pip install bitsandbytes accelerate tokenizers datasets safetensors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 20.9 MB/s eta 0:00:00


In [ ]:
# 깃허브에서 위젯 상태 오류를 피하기 위해 진행 표시줄을 나타내지 않도록 설정합니다.
import os
import tqdm
from transformers.utils import logging

# tqdm 비활성화
tqdm.tqdm = lambda *args, **kwargs: iter([])
tqdm.auto.tqdm = lambda *args, **kwargs: iter([])
tqdm.notebook.tqdm = lambda *args, **kwargs: iter([])
os.environ["DISABLE_TQDM"] = "1"

logging.disable_progress_bar()

### **텍스트 생성 모델 선택하기**

#### **AI 모델 분류**
- **AI 모델 분류 발전사***
|시기|주요 사건|사용된 용어|배경|
|---|---|---|---|
| 2012-2017| 딥러닝 초기| "Proprietary Model"| 기업들이 자체 모델 개발|
| 2018-2019| BERT, GPT-2 등장| "Open vs Closed"| OpenAI가 GPT-2 단계적 공개|
| 2020-2022| **GPT-3, Claude**| "Closed-source Model"| **API 접근만 제공하는 모델** 증가|
| 2023-현재| **LLaMA, Mistral 등**| "Open-weight Model"| **가중치는 공개, 데이터는 비공개**|
- 참고
    - OpenAI GPT-2 Release Strategy (2019): https://openai.com/research/gpt-2-1-5b-release
    - Meta LLaMA Release (2023): https://ai.meta.com/llama/



- **모델 분류 유형**
|용어|한국어|의미|공개 범위|예시|
|---|---|---|---|---|
|Closed-source Model|폐쇄형 모델|모델 가중치, 아키텍처, 학습 데이터 모두 비공개|API만 제공|GPT-4, Claude, Gemini|
|Proprietary Model|독점 모델|상업적 소유권이 있는 모델 (일부는 가중치 공개 가능)|다양함|GPT-4, Claude (완전 비공개)|
|Open-source Model|오픈소스 모델|코드, 가중치, 학습 방법 모두 공개|완전 공개|Pythia, OLMo, BLOOM|
|Open-weight Model|오픈 가중치 모델|가중치는 공개, 학습 데이터/방법은 비공개|가중치만 공개|LLaMA, Mistral, Qwen|
|Open API Model|오픈 API 모델|API로만 접근 가능하지만 무료 또는 저렴|API만|GPT-3.5 (무료 버전)

- 각 **모델 유형의 특징**
|특징|Open-source|Open-weight|Closed-source|
|---|---|---|---|
| 모델 가중치| ✅ 공개| ✅ 공개| ❌ 비공개|
| 학습 코드| ✅ 공개| ⚠️ 부분 공개| ❌ 비공개|
| 학습 데이터| ✅ 공개| ❌ 비공개| ❌ 비공개|
| 아키텍처| ✅ 공개| ✅ 공개| ⚠️ 부분 공개|
| 로컬 실행| ✅ 가능| ✅ 가능| ❌ 불가능|
| 상업적 사용| ✅ 가능 (라이선스에 따라)| ⚠️ 제한적| ❌ API 요금|
| 모델 수정| ✅ 가능| ✅ 가능| ❌ 불가능|
| 재현 가능성| ✅ 높음| ⚠️ 중간| ❌ 불가능|


- **(오픈 소스 모델)파운데이션 모델**(Foundation model)
    - 이런 LLM은 대규모 덱스트 데이터에서 사전 훈련되며 특정 애플리케이션을 위해 미세 튜닝된다.
    - 미세 튜닝된 모델이 수백개 이상 됨
    - 특정 작업에 잘 맞
<img src="https://drive.google.com/uc?export=view&id=1ibK55HC56skczaF9saU9FKlIR6BiZdqT" width="80%">

#### **예제 : 라이선스별 사용 가능 범위 비교**

In [ ]:
# ========================================
# 예제: 라이선스 비교 및 적합성 판단
# ========================================

print("\n" + "="*70)
print("⚖️ AI 모델 라이선스 비교")
print("="*70)

class LicenseChecker:
    """라이선스 적합성 검사기"""

    licenses = {
        "Apache 2.0": {
            "commercial": True,
            "modify": True,
            "redistribute": True,
            "restrictions": [],
            "models": ["Mistral 7B", "Qwen 2.5", "SOLAR"]
        },
        "LLaMA 2 License": {
            "commercial": True,
            "modify": True,
            "redistribute": True,
            "restrictions": ["MAU > 700M requires special license"],
            "models": ["LLaMA 2", "LLaMA 3"]
        },
        "Gemma License": {
            "commercial": True,
            "modify": True,
            "redistribute": True,
            "restrictions": ["Cannot compete with Google services"],
            "models": ["Gemma", "Gemma 2"]
        },
        "GPT-4 API": {
            "commercial": True,
            "modify": False,
            "redistribute": False,
            "restrictions": ["API usage only", "Rate limits", "Cost per token"],
            "models": ["GPT-4", "GPT-4 Turbo"]
        }
    }

    @staticmethod
    def check_use_case(license_name, use_case):
        """사용 사례에 대한 라이선스 적합성 검사"""

        license_info = LicenseChecker.licenses.get(license_name)
        if not license_info:
            return "Unknown license"

        results = []

        # 상업적 사용
        if "commercial" in use_case.lower():
            if license_info["commercial"]:
                results.append("✅ 상업적 사용 가능")
            else:
                results.append("❌ 상업적 사용 불가")

        # 모델 수정
        if "modify" in use_case.lower() or "fine-tun" in use_case.lower():
            if license_info["modify"]:
                results.append("✅ 모델 수정/파인튜닝 가능")
            else:
                results.append("❌ 모델 수정 불가")

        # 재배포
        if "redistribute" in use_case.lower():
            if license_info["redistribute"]:
                results.append("✅ 모델 재배포 가능")
            else:
                results.append("❌ 모델 재배포 불가")

        # 제약사항
        if license_info["restrictions"]:
            results.append(f"⚠️  제약사항:")
            for restriction in license_info["restrictions"]:
                results.append(f"   - {restriction}")

        return results

# 사용 사례 테스트
use_cases = [
    ("Apache 2.0", "스타트업에서 상업적 서비스 개발 및 모델 파인튜닝"),
    ("LLaMA 2 License", "대기업(MAU 10억)에서 상업적 서비스 개발"),
    ("GPT-4 API", "상업적 챗봇 서비스에 API 통합"),
    ("Gemma License", "구글 검색 경쟁 서비스 개발"),
]

print("\n다양한 사용 사례에 대한 라이선스 적합성:\n")

for license_name, use_case in use_cases:
    print(f"【 {license_name} 】")
    print(f"사용 사례: {use_case}")
    print(f"대표 모델: {', '.join(LicenseChecker.licenses[license_name]['models'])}")
    print()

    results = LicenseChecker.check_use_case(license_name, use_case)

    if isinstance(results, list):
        for result in results:
            print(f"  {result}")
    else:
        print(f"  {results}")

    print("-"*70)

print("\n💡 여러분의 프로젝트에 적합한 라이선스를 찾아보세요!")


⚖️ AI 모델 라이선스 비교

다양한 사용 사례에 대한 라이선스 적합성:

【 Apache 2.0 】
사용 사례: 스타트업에서 상업적 서비스 개발 및 모델 파인튜닝
대표 모델: Mistral 7B, Qwen 2.5, SOLAR

----------------------------------------------------------------------
【 LLaMA 2 License 】
사용 사례: 대기업(MAU 10억)에서 상업적 서비스 개발
대표 모델: LLaMA 2, LLaMA 3

  ⚠️  제약사항:
     - MAU > 700M requires special license
----------------------------------------------------------------------
【 GPT-4 API 】
사용 사례: 상업적 챗봇 서비스에 API 통합
대표 모델: GPT-4, GPT-4 Turbo

  ⚠️  제약사항:
     - API usage only
     - Rate limits
     - Cost per token
----------------------------------------------------------------------
【 Gemma License 】
사용 사례: 구글 검색 경쟁 서비스 개발
대표 모델: Gemma, Gemma 2

  ⚠️  제약사항:
     - Cannot compete with Google services
----------------------------------------------------------------------

💡 여러분의 프로젝트에 적합한 라이선스를 찾아보세요!


#### **예제: 모델 비용 계산기**
- 팀 프로젝트시 반드시 해야할 과제임!

In [ ]:
# ========================================
# 예제 : Open vs Closed 모델 비용 비교
# ========================================

print("\n" + "="*70)
print("💰 Open-weight vs Closed-source 비용 비교")
print("="*70)

class ModelCostCalculator:
    """모델 사용 비용 계산기"""

    # API 모델 가격 (2024-2025 기준, USD)
    api_pricing = {
        "GPT-4": {
            "input": 0.03,  # per 1K tokens
            "output": 0.06,
            "type": "closed-source"
        },
        "GPT-3.5 Turbo": {
            "input": 0.0005,
            "output": 0.0015,
            "type": "closed-source"
        },
        "Claude Opus": {
            "input": 0.015,
            "output": 0.075,
            "type": "closed-source"
        },
    }

    # 오픈 모델 인프라 비용 (월 기준)
    open_model_costs = {
        "LLaMA 2 7B": {
            "gpu": "T4",
            "gpu_cost_per_hour": 0.35,  # Google Cloud
            "monthly_cost": 0.35 * 24 * 30,
            "type": "open-weight"
        },
        "Mistral 7B": {
            "gpu": "T4",
            "gpu_cost_per_hour": 0.35,
            "monthly_cost": 0.35 * 24 * 30,
            "type": "open-weight"
        },
        "LLaMA 2 70B": {
            "gpu": "A100",
            "gpu_cost_per_hour": 2.93,
            "monthly_cost": 2.93 * 24 * 30,
            "type": "open-weight"
        },
    }

    @staticmethod
    def calculate_api_cost(model_name, input_tokens, output_tokens):
        """API 모델 비용 계산"""

        pricing = ModelCostCalculator.api_pricing.get(model_name)
        if not pricing:
            return None

        input_cost = (input_tokens / 1000) * pricing["input"]
        output_cost = (output_tokens / 1000) * pricing["output"]
        total_cost = input_cost + output_cost

        return {
            "input_cost": input_cost,
            "output_cost": output_cost,
            "total_cost": total_cost
        }

    @staticmethod
    def compare_costs(monthly_requests, avg_input_tokens, avg_output_tokens):
        """월간 비용 비교"""

        print(f"\n사용량 가정:")
        print(f"  월간 요청 수: {monthly_requests:,}회")
        print(f"  평균 입력 토큰: {avg_input_tokens:,} tokens")
        print(f"  평균 출력 토큰: {avg_output_tokens:,} tokens")
        print(f"  총 토큰/월: {(monthly_requests * (avg_input_tokens + avg_output_tokens)):,} tokens")
        print()

        # API 모델 비용
        print("【 Closed-source (API) 모델 비용 】")
        for model_name, pricing in ModelCostCalculator.api_pricing.items():
            cost_per_request = ModelCostCalculator.calculate_api_cost(
                model_name, avg_input_tokens, avg_output_tokens
            )

            if cost_per_request:
                monthly_cost = cost_per_request["total_cost"] * monthly_requests
                print(f"  {model_name:20s}: ${monthly_cost:,.2f} / 월")

        print()

        # 오픈 모델 비용
        print("【 Open-weight 모델 비용 (인프라) 】")
        for model_name, cost_info in ModelCostCalculator.open_model_costs.items():
            monthly_cost = cost_info["monthly_cost"]
            gpu_type = cost_info["gpu"]
            print(f"  {model_name:20s}: ${monthly_cost:,.2f} / 월 ({gpu_type} 24/7 가동)")

        print()

        # 손익분기점 계산
        print("【 손익분기점 분석 】")

        # GPT-4 vs LLaMA 2 7B
        gpt4_per_request = ModelCostCalculator.calculate_api_cost(
            "GPT-4", avg_input_tokens, avg_output_tokens
        )["total_cost"]

        llama_monthly = ModelCostCalculator.open_model_costs["LLaMA 2 7B"]["monthly_cost"]

        breakeven_requests = llama_monthly / gpt4_per_request

        print(f"  GPT-4 vs LLaMA 2 7B:")
        print(f"    월 {breakeven_requests:,.0f}회 이상 사용 시 LLaMA 2가 경제적")
        print(f"    현재 사용량 ({monthly_requests:,}회): ", end="")

        if monthly_requests > breakeven_requests:
            print("✅ Open-weight 모델이 유리")
        else:
            print("💰 API 모델이 유리")

# 시나리오 테스트
scenarios = [
    {
        "name": "소규모 스타트업",
        "monthly_requests": 10000,
        "avg_input": 500,
        "avg_output": 300
    },
    {
        "name": "중규모 서비스",
        "monthly_requests": 1000000,
        "avg_input": 500,
        "avg_output": 300
    },
    {
        "name": "대규모 서비스",
        "monthly_requests": 10000000,
        "avg_input": 500,
        "avg_output": 300
    }
]

for i, scenario in enumerate(scenarios, 1):
    print("\n" + "="*70)
    print(f"시나리오 {i}: {scenario['name']}")
    print("="*70)

    ModelCostCalculator.compare_costs(
        scenario["monthly_requests"],
        scenario["avg_input"],
        scenario["avg_output"]
    )

print("\n" + "="*70)
print("💡 여러분의 프로젝트 규모에 맞는 모델을 선택해보세요!")
print("="*70)


💰 Open-weight vs Closed-source 비용 비교

시나리오 1: 소규모 스타트업

사용량 가정:
  월간 요청 수: 10,000회
  평균 입력 토큰: 500 tokens
  평균 출력 토큰: 300 tokens
  총 토큰/월: 8,000,000 tokens

【 Closed-source (API) 모델 비용 】
  GPT-4               : $330.00 / 월
  GPT-3.5 Turbo       : $7.00 / 월
  Claude Opus         : $300.00 / 월

【 Open-weight 모델 비용 (인프라) 】
  LLaMA 2 7B          : $252.00 / 월 (T4 24/7 가동)
  Mistral 7B          : $252.00 / 월 (T4 24/7 가동)
  LLaMA 2 70B         : $2,109.60 / 월 (A100 24/7 가동)

【 손익분기점 분석 】
  GPT-4 vs LLaMA 2 7B:
    월 7,636회 이상 사용 시 LLaMA 2가 경제적
    현재 사용량 (10,000회): ✅ Open-weight 모델이 유리

시나리오 2: 중규모 서비스

사용량 가정:
  월간 요청 수: 1,000,000회
  평균 입력 토큰: 500 tokens
  평균 출력 토큰: 300 tokens
  총 토큰/월: 800,000,000 tokens

【 Closed-source (API) 모델 비용 】
  GPT-4               : $33,000.00 / 월
  GPT-3.5 Turbo       : $700.00 / 월
  Claude Opus         : $30,000.00 / 월

【 Open-weight 모델 비용 (인프라) 】
  LLaMA 2 7B          : $252.00 / 월 (T4 24/7 가동)
  Mistral 7B          : $252.00 / 월 (T4 24/7 가동)
  LLaMA 2 70B    

### **텍스트 생성 모델 로드하기**

- 사용 모델 :
    - microsoft/Phi-3-mini-4k-instruct
        - 8GN VRAM  장치에서 실행 가능
    - "skt/KoGPT2-base-v2"
        - SKT가 개발한, 텍스트 생성(Generative)이 가능한 한국어 GPT-2 모델의 기본(base) 버전
        - 약 1억 2500만 개(125M)의 파라미터

코드 짜는거 외우기


In [ ]:
import torch
# AutoModelForCausalLM: 생성모델
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# 모델과 토크나이절르 로드합니다.
model_id = "microsoft/Phi-3-mini-4k-instruct"
# model_id = "beomi/Llama-3-Open-Ko-8B"
# model_id = "Qwen/Qwen2.5-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
    attn_implementation="eager"  # 어텐션 구현 방식을 'eager' (기본)로 명시
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 파이프라인을 만듭니다.
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False, # 프롬프트를 제외하고 생성된 부분만 반환
    max_new_tokens=500,     # 단, KoGPT2를 사용한다면 4k 토큰을 처리하지 못하므로 500은 너무 깁니다. (150 정도로 조절)
    do_sample=False,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


- **호랑이와 관련된 농담을 만들어 달라고 요청하기**

In [ ]:
# 프롬프트
messages = [
    {"role": "user", "content": "호랑이와 관련된 재미난 이야기를 만들어줘."}
] # system, user, assistent 3개가 있음. 프롬프트 구조 공부

# 출력을 생성합니다.
output = pipe(messages)
print(output[0]["generated_text"])

 호랑이와 관련된 재미난 이야기:


저는 호랑이와 관련된 이야기를 생각해 보았습니다. 호랑이는 예술적으로 매우 유명합니다. 그 이유는 그들의 멋진 형태와 느낌입니다. 호랑이는 대형 몸무게, 발길이가 넓은 발, 반지 등 특이한 형태를 가지고 있습니다. 그들은 매우 강력하고 잠실이 되어 있어 예술적으로 높은 감정을 주며 작품을 작성하는 데 도움이 됩니다.


한편, 호랑이는 예술적으로도 잘 작용합니다. 그들의 몸과 몸집을 통해 다양한 예술 방법을 사용합니다. 예를 들어, 그들은 물건을 싸고 있거나 물건을 쓰는 것으로 작품을 만들고 있습니다. 이��


- **템플릿 적용하기**

In [ ]:
# 프롬프트 템플릿을 적용합니다.
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False)
print(prompt)

<|user|>
호랑이와 관련된 재미난 이야기를 만들어줘.<|end|>
<|endoftext|>


### **모델 출력 제어하기**

- **모델 매개변수를 조정하여 출력 제어하기**
    - (참고) PyTorch의 언어 모델 생성 함수에서 사용되는 주요 매개변수.ipynb
    - **temperature** :텍스트 생성의 무작위성(창의성) 조절
    - **do_sample** : 샘플링 방식 선택 (False=Greedy)
    - **top_p** : 뉴클리어 샘플링(nuclear sampling), LLM이 고려할 토큰 일부(뉴클리어스)를 제어하는 샘플링 기법, top_p에 지정한 누적 확률에 도달할 때까지 확률 크기 순으로 토큰을 모음(확률 높은 토큰의 랭킹)

In [ ]:
# 높은 temperature를 사용합니다.
# temperature 값을 조정해 보세요. (0 ~ 2)
#       0 : 수학문제
# 0.3-0.5 : 보고서 작성
#       1 : 기본 대화
# 1.5-2.0 : 소설, 시 창작
output = pipe(messages,
              do_sample=True, # False면 그리디
              temperature=0.0)
print(output[0]["generated_text"])

ValueError: `temperature` (=0.0) has to be a strictly positive float, otherwise your next token scores will be invalid. If you're looking for greedy decoding strategies, set `do_sample=False`.

In [ ]:
# temperature = 1.5
output = pipe(messages,
              do_sample=True, # False면 그리디
              temperature=1.5)
print(output[0]["generated_text"])

 월에같은 요정을 예가 기퀴 중단으로 만든 이유가 아닌 로스그뱃버들이 떠있었다. Roosgevrissen는 아직도 내부에 사의정의를 이룰수도, 나지만 죵중에 기퀴이 예가 터지듯이 산고가 소리를 내리는 것을 유명하게 홑하였다.


그리고 한 구대화에서는 날카로이가 도모로나를 산보트로 보인 Roosgevrissen와 땀볕같은 바다를 한소리를 내며 말같이 한밟풍스츠를 해보았다. 반서에 동시에 호랑이가 들어오며, 근어내려마신사의 줄세를 들며 홀로도 도달한다.


그에 따라 기퀴들이 바람에 내리도록 이바구니를 딱 연주하며 소란들에게의 마음보다 줍리라는 뮤지컬화서는 강하게 들던. 결국, 고유어들도 수직임, 무잠의 상호조화로 요원의 지식 사상을 인정하며 어머녀와 아들들도 서사쌈를 칭뜨겨다


In [ ]:
# 높은 top_p를 사용합니다.
# top_p 값을 조정해 보세요.
output = pipe(messages, do_sample=True, top_p=1)
print(output[0]["generated_text"])

 클래스 종교에 선물로 발송했다. 그래서 우리가 마감이 오기 전에 광고메시지를 텔레비전에 넣었다. 이렇게 한 방송은 어느정도 오셔도 잘들어요. 철수는 항상 좋을 수 있는 행복한 이야기에 잘 들었다. 또한, 우리가 도와주었던 후원자들에게는 사랑스러운 반올림이에요. 이제, 이야기를 찾아가자. 클래스의 아비는 회장이 도와주는 애인들과의 행운을 위해 작은 자리에 항결을 나타낸다. 이 암호를 발견한 후원자들은 모여나온 친구와 앞으로 그들의 기부금을 찾는 것에 집중하게 되었다. 때문에, 그들은 마침내 뒤틀어 대한 재산으로부터 다시 메모장을 벌어 주었다. 이제 윤리적인 대처를 다루고 있는 대학에서는 홍보


### **[실습] 다양한 모델로 텍스트 생성하기**
모델을 변경하여 텍스트를 생성해 보고 생성된 결과의 특징을 확인해 보세요.
1. model_id = "microsoft/Phi-3-mini-4k-instruct"
2. model_id = "beomi/Llama-3-Open-Ko-8B"
3. model_id = "Qwen/Qwen2.5-3B-Instruct"

In [ ]:
import torch
# AutoModelForCausalLM: 생성모델
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# 모델과 토크나이절르 로드합니다.
model_id = "beomi/Llama-3-Open-Ko-8B"
# model_id = "beomi/Llama-3-Open-Ko-8B"
# model_id = "Qwen/Qwen2.5-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
    attn_implementation="eager"  # 어텐션 구현 방식을 'eager' (기본)로 명시
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 파이프라인을 만듭니다.
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False, # 프롬프트를 제외하고 생성된 부분만 반환
    max_new_tokens=500,     # 단, KoGPT2를 사용한다면 4k 토큰을 처리하지 못하므로 500은 너무 깁니다. (150 정도로 조절)
    do_sample=False,
)

# 프롬프트 템플릿을 적용합니다.
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False)
print(prompt)

# 프롬프트
messages = [
    {"role": "user", "content": "호랑이와 관련된 재미난 이야기를 만들어줘."}
] # system, user, assistent 3개가 있음. 프롬프트 구조 공부

# 출력을 생성합니다.
output = pipe(messages)
print(output[0]["generated_text"])

config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/3.00G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/2.94G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/2.97G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Device set to use cuda


<|begin_of_text|><|start_header_id|>user<|end_header_id|>

호랑이와 관련된 재미난 이야기를 만들어줘.<|eot_id|>
호랑이와 관련된 재미난 이야기를 만들어줘.assistant
이번엔 1년 전과는 달리, 1년 후의 상황을 예측하는 데 집중했다. 1년 후의 상황을 예측하는 데는 1년 전의 상황을 예측하는 것보다 훨씬 더 어려운 일이다. 1년 전의 상황을 예측하는 데는 과거의 데이터를 활용할 수 있지만, 1년 후의 상황을 예측하는 데는 과거의 데이터를 활용할 수 없다. 1년 후의 상황을 예측하는 데는 과거의 데이터를 활용할 수 없기 때문에, 1년 후의 상황을 예측하는 것은 1년 전의 상황을 예측하는 것보다 훨씬 더 어려운 일이다. 
 1년 후의 상황을 예측하는 데는 과거의 데이터를 활용할 수 없기 때문에, 1년 후의 상황을 예측하는 것은 1년 전의 상황을 예측하는 것보다 훨씬 더 어려운 일이다. 
 1년 후의 상황을 예측하는 데는 과거의 데이터를 활용할 수 없기 때문에, 1년 후의 상황을 예측하는 것은 1년 전의 상황을 예측하는 것보다 훨씬 더 어려운 일이다. 
 1년 후의 상황을 예측하는 데는 과거의 데이터를 활용할 수 없기 때문에, 1년 후의 상황을 예측하는 것은 1년 전의 상황을 예측하는 것보다 훨씬 더 어려운 일이다. 
 1년 후의 상황을 예측하는 데는 과거의 데이터를 활용할 수 없기 때문에, 1년 후의 상황을 예측하는 것은 1년 전의 상황을 예측하는 것보다 훨씬 더 어려운 일이다. 
 1년 후의 상황을 예측하는 데는 과거의 데이터를 활용할 수 없기 때문에, 1년 후의 상황을 예측하는 것은 1년 전의 상황을 예측하는 것보다 훨씬 더 어려운 일이다. 
 1년 후의 상황을 예측하는 데는 과거의 데이터를 활용할 수 없기 때문에, 1년 후




---



## **고급 프롬프트 엔지니어링**



### **고급 프롬프트 구성 요소**

- **역할/페르소나**(정체성)
    - 모델에게 “누구처럼 행동할지”를 명시합니다
- **지시사항/명령어** (핵심 작업)
    - 프롬프트의 핵심으로, AI에게 무엇을 해야 하는지 명확하게 전달합니다
    - 모호한 표현보다는 구체적이고 직접적인 동사를 사용하는 것이 효과적입니다
    - "해줘", "만들어줘", "분석해줘", "요약해줘" 등의 명확한 동사 사용
- **문맥/맥락/컨텍스트** (추가 정보)
    - AI가 작업을 수행할 때 필요한 추가 정보/배경 정보를 제공합니다
    - 대상 독자, 목적, 상황 등을 포함하면 더 적절한 응답을 얻을 수 있습니다
- **형식** (추가 정보)
    - LLM이 생성한 텍스트를 출력하는 데 사용할 형식. 이를 지정하지 않으면 LLM이 스스로 형식을 결정하기 때문에 자동화된 시스템에서 문제가 됩니다.
- **청중**
    - 생성된 텍스트의 소비 대상, 생성된 출력의 수준도 기술합니다.
    - 교육이 목적이라면 ELI5 (Explain Like I'm 5: "5살 아이에게 설명하듯이 쉽게 설명해주세요"라는 뜻)를 사용하는게 도움이 됩니다.
- **어투**
    - LLM이 생성된 텍스트에서 사용할 말투, 상사에게 업무 메일을 쓴다면 격식을 차린 어투가 필요할 것입니다.
- **데이터**
    - 작업 자체에 관련된 주요 데이터

### **복잡한 프롬프트**

- 최상의 프롬프트를 얻으려면 실험이 필수이다.
- 프롬프트 구성은 기본적으로 반복적인 실험 과정이다.

In [ ]:
text = """이전 게시물에서는 최신 딥러닝 모델에서 널리 사용되는 방법인 어텐션(Attention)을 살펴보았습니다. 어텐션은 신경망 기계 번역 애플리케이션의 성능 향상에 기여한 개념입니다. 이번 게시물에서는 어텐션을 사용하여 모델의 학습 속도를 높이는 모델인 트랜스포머(The Transformer)를 살펴보겠습니다. 트랜스포머는 특정 작업에서 구글 신경망 기계 번역 모델보다 우수한 성능을 보입니다. 하지만 가장 큰 장점은 트랜스포머가 병렬화에 적합하다는 점입니다. 실제로 구글 클라우드는 자사의 클라우드 TPU 솔루션을 사용하기 위해 트랜스포머를 참조 모델로 사용할 것을 권장합니다. 이제 모델을 분해하여 어떻게 작동하는지 살펴보겠습니다.
트랜스포머는 "Attention is All You Need" 논문에서 제안되었습니다. 텐서플로우(TensorFlow) 기반 구현은 Tensor2Tensor 패키지의 일부로 제공됩니다. 하버드 대학교 자연어 처리 그룹에서는 PyTorch 구현을 사용하여 논문에 주석을 단 가이드를 작성했습니다. 이 글에서는 내용을 다소 단순화하고 개념을 하나씩 소개하여 해당 주제에 대한 심층적인 지식이 없는 사람들도 더 쉽게 이해할 수 있도록 하겠습니다.
이 모델을 하나의 블랙박스로 생각해 보겠습니다. 기계 번역 애플리케이션에서 이 모델은 한 언어로 된 문장을 입력받아 다른 언어로 번역된 결과를 출력합니다.
옵티머스 프라임처럼 생긴 이 모델을 열면 인코딩 구성 요소, 디코딩 구성 요소, 그리고 이들 간의 연결을 볼 수 있습니다.
인코딩 구성 요소는 인코더의 스택입니다(이 논문에서는 인코더를 여섯 개씩 쌓아 올렸는데, 숫자 6이 마법 같은 것은 아니며, 다른 배열을 시도해 볼 수 있습니다). 디코딩 구성 요소는 같은 수의 디코더의 스택입니다.
인코더는 모두 구조가 동일하지만 가중치는 공유하지 않습니다. 각 인코더는 두 개의 하위 계층으로 나뉩니다.
인코더의 입력은 먼저 셀프 어텐션 계층을 통과합니다. 셀프 어텐션 계층은 인코더가 특정 단어를 인코딩할 때 입력 문장의 다른 단어들을 살펴보는 데 도움이 되는 계층입니다. 이 글의 후반부에서 셀프 어텐션에 대해 자세히 살펴보겠습니다.
셀프 어텐션 계층의 출력은 피드포워드 신경망에 입력됩니다. 동일한 피드포워드 신경망이 각 위치에 독립적으로 적용됩니다.
디코더는 두 계층을 모두 가지고 있지만, 두 계층 사이에는 디코더가 입력 문장의 관련 부분에 집중할 수 있도록 돕는 어텐션 계층이 있습니다(seq2seq 모델에서 어텐션이 하는 역할과 유사).
이제 모델의 주요 구성 요소를 살펴보았으니, 다양한 벡터/텐서와 이러한 벡터/텐서가 구성 요소 사이를 어떻게 흐르면서 학습된 모델의 입력을 출력으로 변환하는지 살펴보겠습니다.
일반적인 NLP 애플리케이션과 마찬가지로, 임베딩 알고리즘을 사용하여 각 입력 단어를 벡터로 변환하는 것으로 시작합니다.
각 단어는 크기가 512인 벡터에 임베딩됩니다. 이러한 벡터를 다음과 같은 간단한 상자로 표현하겠습니다.
임베딩은 가장 아래쪽 인코더에서만 수행됩니다. 모든 인코더에 공통적인 추상화는 각각 크기가 512인 벡터 목록을 받는다는 것입니다. 맨 아래 인코더에서는 단어 임베딩이 되고, 다른 인코더에서는 바로 아래에 있는 인코더의 출력이 됩니다. 이 목록의 크기는 우리가 설정할 수 있는 하이퍼파라미터로, 기본적으로 학습 데이터 세트에서 가장 긴 문장의 길이가 됩니다.
입력 시퀀스에 단어를 임베딩한 후, 각 단어는 인코더의 두 계층을 각각 통과합니다.
여기서 우리는 Transformer의 핵심 속성 중 하나를 발견하게 되는데, 각 위치의 단어는 인코더에서 자체 경로를 따라 흐른다는 것입니다. 셀프 어텐션 계층에서는 이러한 경로 간에 종속성이 있습니다. 그러나 피드포워드 계층에는 이러한 종속성이 없으므로, 피드포워드 계층을 통과하는 동안 다양한 경로가 병렬로 실행될 수 있습니다.
다음으로, 예시를 더 짧은 문장으로 전환하여 인코더의 각 하위 계층에서 어떤 일이 일어나는지 살펴보겠습니다.
이제 인코딩을 시작합니다!
앞서 언급했듯이, 인코더는 벡터 목록을 입력으로 받습니다. 인코더는 이 벡터들을 '셀프 어텐션' 계층으로 전달한 후, 피드포워드 신경망으로 전달하여 목록을 처리한 후, 출력을 다음 인코더로 전송합니다."""

# 프롬프트 구성 요소
persona = "당신은 대규모 언어 모델 전문가입니다. 복잡한 논문을 이해하기 쉬운 요약으로 정리하는 데 능숙합니다.\n"
instruction = "제공된 논문의 주요 결과를 요약하세요.\n"
context = "귀하의 요약에서는 연구자들이 논문의 가장 중요한 정보를 빠르게 이해하는 데 도움이 되는 가장 중요한 요점을 추출해야 합니다.\n"
data_format = "방법을 간략하게 요약한 요점 요약을 작성하세요. 그 후 주요 결과를 요약하는 간결한 단락으로 마무리하세요.\n"
audience = "이 요약은 대규모 언어 모델의 최신 동향을 빠르게 파악해야 하는 바쁜 연구자들을 위해 고안되었습니다.\n"
tone = "톤은 전문적이고 명확해야 합니다.\n"
text = "요약할 내 텍스트"  # Replace with your own text to summarize
data = f"요약할 텍스트: {text}"

# 전체 프롬프트 - 요소를 삭제하거나 추가하여 생성된 출력에 미치는 영향을 관찰하세요.
query = persona + instruction + context + data_format + audience + tone + data

In [ ]:
messages = [
    {"role": "user", "content": query}
]
print(tokenizer.apply_chat_template(messages, tokenize=False))

<|user|>
당신은 대규모 언어 모델 전문가입니다. 복잡한 논문을 이해하기 쉬운 요약으로 정리하는 데 능숙합니다.
제공된 논문의 주요 결과를 요약하세요.
귀하의 요약에서는 연구자들이 논문의 가장 중요한 정보를 빠르게 이해하는 데 도움이 되는 가장 중요한 요점을 추출해야 합니다.
방법을 간략하게 요약한 요점 요약을 작성하세요. 그 후 주요 결과를 요약하는 간결한 단락으로 마무리하세요.
이 요약은 대규모 언어 모델의 최신 동향을 빠르게 파악해야 하는 바쁜 연구자들을 위해 고안되었습니다.
톤은 전문적이고 명확해야 합니다.
요약할 텍스트: 요약할 내 텍스트<|end|>
<|endoftext|>


In [ ]:
# 출력을 생성합니다.
outputs = pipe(messages)
print(outputs[0]["generated_text"])

 요약할 내 텍스트: 이 논문은 언어 모델의 학습 과정에 대한 더 잘 이해된 이론을 제시하고 있습니다. 논문은 언어 모델이 언어 표현을 학습하는 방식에 대한 더 잘 이해된 이론을 제시하고 있습니다. 이러한 이론은 언어 모델이 언어 표현을 학습하는 방식에 대한 더 잘 이해된 이론을 제시하고 있습니다. 본 논문은 언어 모델이 언어 표현을 학습하는 방식에 대한 더 잘 이해된 이론을 제시하고 있습니다. 이러한 이론은 언어 모델이 언어 표현을 학습하는 방식에 대한 더 잘 이해된 이론을 제시하고 있습니다. 본 논문은 언어 모델이 언어 표현을 학습하는 방식에 대한 더 잘 이해된 이론을 제시하고 있습니다. 이러한 이론은


In [ ]:
import gc
import torch

# 캐시된 메모리 해제
torch.cuda.empty_cache()

# 가비지 컬렉션 실행
gc.collect()

print("GPU 메모리 클리어 완료!")

GPU 메모리 클리어 완료!


### **문맥 내 학습: 예시 제공**

- **문맥 내 학습**(in-context learning) : 모델에게 올바른 예시를 제공하는 방법
    - **제로샷**(zero-shot) 프롬프트 : 예시를 활용하지 않는다.
    - **원샷**(one-shot) 프롬프트 : 한 개 예시 사용
    - **퓨샷**(few-shot) 프롬프트 : 두 개 이상의 예시 사용

- 제로샷: 예시 0개
    - user
- 원샷: 예시 1개
    - user → assistant → user
- 퓨샷: 예시 2개 이상
    - (user → assistant) × N → user

In [ ]:
# 문장에 가상의 단어가 포함된 예시를 사용합니다.
one_shot_prompt = [
    {
        "role": "user",
        "content": "'기가무루'는 일본 악기의 한 종류입니다. '기가무루'라는 단어를 사용한 문장의 예는 다음과 같습니다.:"
    },
    {
        "role": "assistant",
        "content": "삼촌이 선물해 주신 기가무루가 있어요. 집에서 하는 걸 좋아해요.:"
    },
    {
        "role": "user",
        "content": "무언가를 'screeg'한다는 것은 칼을 휘두르는 것을 의미합니다. screeg라는 단어를 사용한 문장의 예는 다음과 같습니다.:"
    }
]
print(tokenizer.apply_chat_template(one_shot_prompt, tokenize=False))

<|user|>
'기가무루'는 일본 악기의 한 종류입니다. '기가무루'라는 단어를 사용한 문장의 예는 다음과 같습니다.:<|end|>
<|assistant|>
삼촌이 선물해 주신 기가무루가 있어요. 집에서 하는 걸 좋아해요.:<|end|>
<|user|>
무언가를 'screeg'한다는 것은 칼을 휘두르는 것을 의미합니다. screeg라는 단어를 사용한 문장의 예는 다음과 같습니다.:<|end|>
<|endoftext|>


In [ ]:
# 출력을 생성합니다.
outputs = pipe(one_shot_prompt)
print(outputs[0]["generated_text"])

 저는 칼을 휘두르는 것을 즐겨하고 있어요. 칼을 휘두르면 나무를 덮어 잡고 있어요. 칼을 휘두르는 것은 잘 잘 되어 있는 나무를 잡아 내리는 것이 매우 힘듭니다. 그래서 나는 칼을 휘두르는 것을 좋아합니다. 그러나 칼을 휘두르는 것은 약속을 못 외롭게 하거나 칼을 쓰는 것을 잊지 마셔야 합니다. 칼을 휘두르는 것은 잘 잘 잡아 내리는 것이 아니고 잘 잘 잡아 더러운 것입니다. 그래서 칼을 휘두르는 것은 잘 잘 잡아 더러운 나무를 잡아 내리는 것이 매우 힘듭니다. 그래서 나는 칼을 휘두르는 것을 좋아합니다. 그러나 칼을 ��������


### **프롬프트 체인: 문제 쪼개기**

- 여러 프롬프트로 분할하기
- **한 프롬프트의 출력을 다음 프롬프트의 입력으로 사용**하는 식으로 연속적인 상호작용 체인을 만들어 문제를 해결한다. --> **순차적인 파이프라인 생성**
- 각각의 호출에 파이프 매개변수를 다르게 지정할 수 있다는 것이 장점
-  사용 사례 예:
    - **응답 유형성 검사** :
        - 이전에 생성한 출력을 제확인하도록 LLM에게 요청한다.
    - **병렬 프롬프트** :  
        - 여러 개의 프롬프트를 병렬로 만들고 최종 단계에서 병합한다. 예를 들어 복수의 LLM에게 여러 개의 레시피를 병렬로 생성하도록 요청한다.
        - 그 다음 이 결과를 합쳐서 쇼핑 목록을 만들도록 한다.
    - **이야기 작성** :
        - LLM을 활용하여 문제를 여러 요소로 나누는 방식을 사용해 책이나 이야기를 작성한다.
        - 예를 들어, 먼저 요약을 작성하고, 캐릭터를 개발하고, 핵심 장면을 만들고, 그 다음 대화를 만드는 단계로 넘어간다.


### 🆚 비교: 체인 vs 한 번에

#### 한번에 (단점)
-  모든 단계에 같은 매개변수 적용
result = pipe(
    "아이디어 내고, 분석하고, 홍보 문구 만들어줘",
    temperature=0.7,  # 모든 단계에 동일!
    max_new_tokens=500
)

→ 창의적인 부분은 부족하고, 정확해야 할 부분은 불안정

#### 체인 (장점)
- 각 단계마다 최적 매개변수
idea = pipe(..., temperature=1.2)      # 창의적
analysis = pipe(..., temperature=0.3)  # 정확
promo = pipe(..., temperature=0.9)     # 다시 창의적

→ 각 단계의 목적에 최적화!


---

## 매개변수 선택 가이드

| 작업 유형 | temperature | max_tokens | top_p |
|----------|-------------|------------|-------|
| **사실 확인** | 0.1-0.3 | 짧게 | 0.8 |
| **분류/분석** | 0.2-0.4 | 짧게 | 0.85 |
| **요약** | 0.3-0.5 | 짧게 | 0.9 |
| **설명** | 0.5-0.7 | 중간 | 0.9 |
| **창작/마케팅** | 0.8-1.2 | 길게 | 0.95 |
| **브레인스토밍** | 1.0-1.5 | 중간 | 0.95 |

---

## 정리

**매개변수 = LLM 생성 설정값**

- `temperature`: 창의성
- `max_new_tokens`: 길이
- `top_p`: 품질/다양성
- `do_sample`: 확률적/결정적

**체인의 장점 = 단계마다 최적 설정 가능**

- 창의적 단계 → temperature 높게

- 정확한 단계 → temperature 낮게

- 짧은 출력 → max_tokens 적게

- 긴 출력 → max_tokens 많게

In [ ]:
# 제품 이름과 슬로건을 만듭니다.
product_prompt = [
    {"role": "user", "content": "한국 여자 아이 이름 한 개를 지어줘."}
]
outputs = pipe(product_prompt)
product_description = outputs[0]["generated_text"]
print(product_description)

 민지


In [ ]:
# 제품 이름과 슬로건을 바탕으로 홍보 문구를 생성합니다.
sales_prompt = [
    {"role": "user", "content": f"다음 사람에 대한 홍보 문구를 생성합니다. : '{product_description}'"}
]
outputs = pipe(sales_prompt)
sales_pitch = outputs[0]["generated_text"]
print(sales_pitch)

 민지 소개

민지는 현대 기술에 맞춰 노력하는 인물입니다. 저는 컴퓨터 과학, 디자인, 및 프로그래밍 분야에서 많은 경력을 가지고 있습니다. 저의 경험은 컴퓨터 프로그래밍 업무에서 첫 번째로 작성한 코드 프로젝트에 대한 성공적인 경험을 있습니다. 또한, 저는 프로젝트 운영, 프로그래밍 실습, 그리고 프로젝트 프로토타입 등의 분야에서 많은 경험을 가지고 있습니다. 저의 목표는 새로운 기술을 개발하고, 기업에서 효율적인 프로그래밍 업무를 수행하는 데 도움을 줄 것입니다.




---



## **생성 모델을 사용한 추론**


### **CoT**(**Chain-of-thought**): **응답하기 전에 생각하기**

- **CoT(<mark>**Chain-of-thought**</mark>)의 목표**
    - **생성 모델이 추론 과정 없이 바로 질문에 대답하지 않고 그 전에 생각하게 만드는 것**

In [ ]:
# 추론없이 답변하기
standard_prompt = [
    {"role": "user", "content": "로저는 테니스공 5개를 가지고 있습니다. 그는 테니스공 캔 두 개를 더 삽니다. 각 캔에는 테니스공이 3개씩 들어 있습니다. 이제 그는 테니스공을 몇 개 가지고 있습니까?"},
    {"role": "assistant", "content": "11"},
    {"role": "user", "content": "식당에는 사과 23개가 있었습니다. 점심을 만드는데 사과를 20개를 사용하고 6개를 더 샀다면 남은 사과는 몇 개입니까?"}
]

# 출력을 생성합니다.
outputs = pipe(standard_prompt) #standard_prompt : 추론 능력 x -> 오답 가능성 높음
print(outputs[0]["generated_text"])

 식당에서 점심을 만들 때 사과를 사용했습니다. 총 사과 23개 중에서 사과를 20개 사용했습니다. 이 값을 식당에서 있던 사과 23개 빼면 남은 사과는 3개입니다. 그런데 식당에서 6개를 더 샀다면 이 3개가 6개로 증가할 것입니다. 따라서 식당에서 사과가 6개 더 있습니다.


In [ ]:
# CoT로 대답하기
cot_prompt = [
    {"role": "user", "content": "로저는 테니스공 5개를 가지고 있습니다. 그는 테니스공 캔 두 개를 더 삽니다. 각 캔에는 테니스공이 3개씩 들어 있습니다. 이제 그는 테니스공을 몇 개 가지고 있습니까?"},
    {"role": "assistant", "content": "로저는 공 5개로 시작했습니다. 테니스공 3개가 들어있는 캔 2개는 테니스공 6개입니다. 5 + 6 = 11입니다. 답은 11입니다."},
    {"role": "user", "content": "식당에는 사과 23개가 있었습니다. 점심을 만드는데 사과를 20개를 사용하고 6개를 더 샀다면 남은 사과는 몇 개입니까?"}
]

# 출력을 생성합니다.
outputs = pipe(cot_prompt) #cot_prompt : 생성모델이 생각하게 만드는 것. 추론 능력 덕분에 새로운 문제에도 대답 가능
print(outputs[0]["generated_text"])

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


 식당에서 점심을 만들 때 사과를 사용했습니다. 총 사과 23개 중에서 사과를 20개 사용했습니다. 남은 사과는 23 - 20 = 3개입니다. 또한 6개를 더 샀기 때문에 3개 + 6개 = 9개가 남았습니다. 답은 9개입니다.


### **제로-샷 CoT**



In [ ]:
# 제로-샷 CoT
zeroshot_cot_prompt = [
    {"role": "user", "content": "식당에는 사과 23개가 있었습니다. 만약 20개를 점심을 만드는데 사용하고 6개를 더 샀다면, 남은 사과는 몇 개일까요? 단계별로 생각해 봅시다."}
]

# 출력을 생성합니다.
outputs = pipe(zeroshot_cot_prompt)
print(outputs[0]["generated_text"])

 1. 식당에 있었던 사과의 수: 23개

2. 점심을 만드는 사과의 수: 20개

3. 새로운 사과 추가: 6개

4. 총 사과 수: 20개 + 6개 = 26개

5. 남은 사과 수: 23개 - 26개 = -3개


따라서, 남은 사과는 음수가 나온 것으로 봤습니다. 이는 사과가 없는 상황을 나타냅니다. 즉, 사과가 없습니다.


###**자기 일관성**(**self-consistency**): 출력 샘플링
  - 무작위성에 대응하고 생성 모델의 품질을 향상시키기 시키기 위해 자기 일관성 방법이 개발됨
  - **생성 모델에 동일한 프롬프트를 여러 번 요청**하고 다수를 차지하는 결과를 최종 답변으로 내놓는 방법
  - 이 과정에서 샘플링의 다양성을 증가시키기 위해 서로 다른 temperature와 top_p 값을 사용해 답변을 생성할 수 있다.

In [ ]:
import re
import math
import random
import torch
from collections import Counter

def format_messages(messages, tokenizer):
    """chat 템플릿 적용"""
    try:
        return tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
    except Exception:
        text = ""
        for m in messages:
            role = m.get("role", "user")
            content = m.get("content", "")
            text += f"[{role.upper()}]\n{content}\n\n"
        text += "[ASSISTANT]\n"
        return text

def normalize_answer(ans):
    """답변 정규화"""
    korean = {"하나": "1", "둘": "2", "셋": "3", "넷": "4",
              "다섯": "5", "여섯": "6", "일곱": "7", "여덟": "8", "아홉": "9"}
    for k, v in korean.items():
        ans = ans.replace(k, v)

    nums = re.findall(r'\d+', ans.replace(",", ""))
    return nums[0] if nums else ans.strip()

num_pattern = re.compile(r"(-?\d+(?:[.,]\d+)?)")

def extract_answer(text):
    """생성 결과에서 답변 추출 + 정규화"""
    # "답은 X" 패턴
    m = re.search(r"답\s*[:은]\s*([^\n\.]+)", text)
    if m:
        cand = m.group(1).strip()
        return normalize_answer(cand)

    # 숫자 탐색
    nums = num_pattern.findall(text.replace(",", ""))
    if nums:
        return normalize_answer(nums[-1])

    # 마지막 줄
    last_line = text.strip().splitlines()[-1]
    return normalize_answer(last_line.strip())

def remove_outliers(candidates):
    """이상치 제거 (선택사항)"""
    try:
        nums = [float(c) for c in candidates
                if c.replace('.','').replace('-','').isdigit()]
        if len(nums) > 3:
            import statistics
            mean = statistics.mean(nums)
            stdev = statistics.stdev(nums)

            filtered = []
            for c in candidates:
                if not c.replace('.','').replace('-','').isdigit():
                    filtered.append(c)
                elif abs(float(c) - mean) <= 2 * stdev:
                    filtered.append(c)
            return filtered
    except:
        pass
    return candidates

def self_consistent_generate(
    messages,
    n_samples=12,
    temperatures=(0.4, 0.7, 0.9, 1.1),
    top_ps=(0.7, 0.9, 0.95),
    max_new_tokens=128,
    seed=42,
    use_outlier_removal=False,
):
    """
    개선된 자기 일관성 생성
    """
    prompt = format_messages(messages, tokenizer)
    candidates = []
    meta = []

    combos = [(t, p) for t in temperatures for p in top_ps]
    random.seed(seed)
    chosen = [random.choice(combos) for _ in range(n_samples)]

    for i, (t, p) in enumerate(chosen):
        torch.manual_seed(seed + i)
        out = pipe(
            prompt,
            do_sample=True,
            temperature=float(t),
            top_p=float(p),
            max_new_tokens=max_new_tokens,
            eos_token_id=tokenizer.eos_token_id,
        )[0]["generated_text"]

        ans = extract_answer(out)
        candidates.append(ans)
        meta.append({
            "temperature": t,
            "top_p": p,
            "raw": out,
            "parsed": ans
        })

    # 이상치 제거 (선택)
    if use_outlier_removal:
        candidates = remove_outliers(candidates)

    # 다수결
    tally = Counter(candidates)
    best_ans, best_cnt = tally.most_common(1)[0]
    confidence = best_cnt / n_samples

    return {
        "majority_answer": best_ans,
        "vote_count": best_cnt,
        "confidence": f"{confidence:.1%}",
        "confidence_level": "높음" if confidence > 0.7 else "중간" if confidence > 0.5 else "낮음",
        "total_samples": n_samples,
        "votes": dict(tally),
        "samples_meta": meta,
    }

# 개선된 프롬프트
cot_prompt = [
    {"role": "user", "content": "로저는 테니스공 5개를 가지고 있습니다. 그는 테니스공 캔 두 개를 더 삽니다. 각 캔에는 테니스공이 3개씩 들어 있습니다. 이제 그는 테니스공을 몇 개 가지고 있습니까?"},
    {"role": "assistant", "content": "로저는 공 5개로 시작했습니다. 테니스공 3개가 들어있는 캔 2개는 테니스공 6개입니다. 5 + 6 = 11입니다. 답은 11입니다."},
    {"role": "user", "content": "식당에는 사과 23개가 있었습니다. 점심을 만드는 데 20개를 사용하고 6개를 더 샀다면 사과는 몇 개입니까? 위 예시처럼 단계별로 계산하고 마지막에 '답은 X'로 답하세요."},
     # 코틀릿(CoT) 유도 힌트: "생각을 단계적으로 설명한 다음 '답은 X'로 끝내라"
    {"role": "assistant", "content": "생각을 단계적으로 설명한 다음 마지막 줄에 '답은 (정답)' 형태로 끝내세요."}
]

result = self_consistent_generate(
    messages=cot_prompt,
    n_samples=15,
    temperatures=(0.4, 0.7, 1.0),
    top_ps=(0.85, 0.9, 0.95),
    max_new_tokens=120,
    seed=123,
    use_outlier_removal=True
)

print("=== Self-Consistency 결과 (개선 버전) ===")
print(f"최종 답: {result['majority_answer']}")
print(f"득표: {result['vote_count']}/{result['total_samples']}회")
print(f"신뢰도: {result['confidence']} ({result['confidence_level']})")
print(f"득표 분포: {result['votes']}")

=== Self-Consistency 결과 (개선 버전) ===
최종 답: 6
득표: 9/15회
신뢰도: 60.0% (중간)
득표 분포: {'9': 3, '3': 2, '6': 9}


In [ ]:
import re
import math
import random
import torch
from collections import Counter

def format_messages(messages, tokenizer):
    """
    chat 템플릿이 있으면 사용, 없으면 단순 문자열 포맷으로 fallback.
    """
    try:
        return tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
    except Exception:
        # 간단 포맷: role과 content를 순서대로 이어붙임
        text = ""
        for m in messages:
            role = m.get("role", "user")
            content = m.get("content", "")
            text += f"[{role.upper()}]\n{content}\n\n"
        text += "[ASSISTANT]\n"
        return text

# 2) 후보 생성 + 정답 추출 유틸
num_pattern = re.compile(r"(-?\d+(?:[.,]\d+)?)")
def extract_answer(text):
    """
    생성 결과에서 '최종 답'을 추려냅니다.
    - '답은 X' 같은 패턴 우선
    - 숫자 없으면 마지막 줄을 트림
    """
    # 우선 '답은' 패턴 탐색
    m = re.search(r"답\s*[:은]\s*([^\n\.]+)", text)
    if m:
        cand = m.group(1).strip()
        # cand에 숫자가 있으면 그 숫자, 아니면 cand 전체
        m2 = num_pattern.search(cand.replace(",", ""))
        return m2.group(1) if m2 else cand

    # 숫자 전반 탐색
    nums = num_pattern.findall(text.replace(",", ""))
    if nums:
        return nums[-1]  # 마지막으로 언급한 수를 채택(경험상 최종 계산치일 가능성↑)

    # 숫자가 전혀 없으면 마지막 줄
    last_line = text.strip().splitlines()[-1]
    return last_line.strip()

def self_consistent_generate(
    messages,
    n_samples=12,
    temperatures=(0.4, 0.7, 0.9, 1.1),
    top_ps=(0.7, 0.9, 0.95),
    max_new_tokens=128,
    seed=42,
):
    """
    다양한 temperature/top_p 조합으로 n_samples개 생성하여 다수결.
    """
    prompt = format_messages(messages, tokenizer)
    candidates = []
    meta = []

    # 조합 풀 만들기
    combos = [(t, p) for t in temperatures for p in top_ps]
    # n_samples에 맞게 랜덤 샘플링(중복 허용)
    random.seed(seed)
    chosen = [random.choice(combos) for _ in range(n_samples)]

    for i, (t, p) in enumerate(chosen):
        torch.manual_seed(seed + i)
        out = pipe(
            prompt,
            do_sample=True,
            temperature=float(t),
            top_p=float(p),
            max_new_tokens=max_new_tokens,
            eos_token_id=tokenizer.eos_token_id,
        )[0]["generated_text"]

        ans = extract_answer(out)
        candidates.append(ans)
        meta.append({"temperature": t, "top_p": p, "raw": out, "parsed": ans})

    # 다수결
    tally = Counter(candidates)
    best_ans, best_cnt = tally.most_common(1)[0]

    return {
        "majority_answer": best_ans,
        "vote_count": best_cnt,
        "total_samples": n_samples,
        "votes": tally,
        "samples_meta": meta,
    }

# 3) 예시: CoT + Self-Consistency 프롬프트
#    - 첫 번째 QA로 CoT 풀이를 보여준 뒤, 두 번째 문제를 질의
cot_prompt = [
    {"role": "user", "content": "로저는 테니스공 5개를 가지고 있습니다. 그는 테니스공 캔 두 개를 더 삽니다. 각 캔에는 테니스공이 3개씩 들어 있습니다. 이제 그는 테니스공을 몇 개 가지고 있습니까?"},
    {"role": "assistant", "content": "로저는 공 5개로 시작했습니다. 테니스공 3개가 들어있는 캔 2개는 테니스공 6개입니다. 5 + 6 = 11입니다. 답은 11입니다."},
    {"role": "user", "content": "식당에는 사과 23개가 있었습니다. 점심을 만드는 데 20개를 사용하고 6개를 더 샀다면 사과는 몇 개입니까?"},
    # 코틀릿(CoT) 유도 힌트: "생각을 단계적으로 설명한 다음 '답은 X'로 끝내라"
    {"role": "assistant", "content": "생각을 단계적으로 설명한 다음 마지막 줄에 '답은 (정답)' 형태로 끝내세요."}
]

result = self_consistent_generate(
    messages=cot_prompt,
    n_samples=15,                # 표본 수(증가 시 안정성↑, 시간↑)
    temperatures=(0.4, 0.7, 1.0),
    top_ps=(0.85, 0.9, 0.95),
    max_new_tokens=120,
    seed=123
)

print("=== Self-Consistency 결과 ===")
print("다수결 최종 답:", result["majority_answer"])
print("득표수/총샘플:", result["vote_count"], "/", result["total_samples"])
print("득표 분포:", result["votes"])

=== Self-Consistency 결과 ===
다수결 최종 답: 6
득표수/총샘플: 7 / 15
득표 분포: Counter({'6': 7, '9': 3, '3': 2, '29': 1, '2': 1, '23': 1})


### **ToT: 중간 단계 탐색**
- 아이디어를 깊게 탐색할 수 있는 ToT(Tree-of-thought)
- 동작 방법
    - 여러 단계의 추론이 필요한 문제를 만났을 때 이 문제를 여러 단계로 나누고
    - 각 단계에서 생성 모델이 당면한 문제를 위한 여러 다른 솔루션을 탐색하여
    - 최상의 솔루션을 뽑고 다음 단계로 계속 이어가는 방법


In [ ]:
# 제로-샷 ToT
zeroshot_tot_prompt = [
    {"role": "user", "content": "세 명의 전문가가 이 질문에 답한다고 가정해 보세요. 모든 전문가는 자신의 생각의 한 단계를 적어 그룹원들과 공유합니다. 그런 다음 모든 전문가가 다음 단계로 넘어가는 식으로 진행합니다. 어느 시점에서든 자신이 틀렸다는 것을 깨닫는 전문가가 있으면 그 자리에서 나갑니다. 질문은 \"식당에는 사과가 23개 있었습니다. 20개를 점심으로 만들고 6개를 더 샀다면, 그들은 사과를 몇 개 가지고 있을까요?\"입니다. 결과에 대해 반드시 토론하세요."}
]

In [ ]:
# 출력을 생성합니다.
outputs = pipe(zeroshot_tot_prompt)
print(outputs[0]["generated_text"])

 이 문제는 숫자 덧셈 문제입니다. 질문에서 주어진 정보에 따라 답을 찾아야 합니다. 


- 식당에 있었던 사과의 수: 23개

- 점심으로 만들었던 사과의 수: 20개

- 더 샀던 사과의 수: 6개


따라서, 점심으로 만들었던 사과의 수를 더한 사과의 수는 20개 + 6개 = 26개입니다. 따라서 식당에 있었던 사과의 수는 23개 + 6개 = 29개입니다.


따라서 식당에 있었던 사과는 29개, 점심으로 만들었던 사과는 26개입니다.




---



## **출력 검증**
- 생성 모델로 만든 시스템과 애플리케이션은 제품에 투입될 수 있다.
- 애플리케이션이 고장 나는 것을 막고 안정된 생성AI 애플리케이션을 만들기 위해서는 모델의 출력을 검증하고 제어하는 것이 중요하다.
- **출력을 검증하는 이유**
    - **구조적인 출력** : 대부분의 생성모델은 자유로운 형식 텍스트를 만들지만 일부 사용 사례에서는 JSON 같은 특정 포맷의 구조를 가진 출력이 필요하다.
    - **유효한 출력** : 예를 들어 둘 중 하나를 선택하여 출력하라고 요청했을 때 모델이 다른 것을 선택하면 안된다.
    - **윤리** : 욕설, 개인 식별 정보, 편향, 문화적 고정 관념등이 출력에 포함되지 않아야 한다.
    - **정확성** : 특정 표준이나 성능을 따라야 한다. 생성된 정보가 사실적으로 정확하고 일관성이 있고, 환곽이 없는지 재확인해야 한다.
- **출력을 제어하는 방법 3가지**
    - **예시** : 기대하는 출력의 예시를 여러 개 제공한다.
    - **문법** : 토큰 선택 과정을 제어한다.
    - **미세 튜닝** :  기대 출력이 포함된 데이터에서 모델을 튜닝한다.

### **예시 제공**

In [ ]:
# 제로-샷 학습: 예시 없음
zeroshot_prompt = [
    {"role": "user", "content": "JSON 형식으로 RPG 게임의 캐릭터 프로필을 만듭니다."}
]

# 출력을 생성합니다.
outputs = pipe(zeroshot_prompt)
print(outputs[0]["generated_text"])

 ```json

{

  "name": "Alden",

  "class": "Warrior",

  "level": 1,

  "stats": {

    "strength": 15,

    "dexterity": 10,

    "constitution": 14,

    "intelligence": 8,

    "wisdom": 12,

    "charisma": 10

  },

  "equipment": {

    "weapon": "Sword of Valor",

    "armor": "Leather Tunic",

    "accessories": ["Healing Potion", "Shield of Fortitude"]

  },

  "skills": {

    "combat": 8,

    "stealth": 5,

    "alchemy": 3,

    "riddle": 4

  },

  "background": "Alden was a blacksmith's apprentice who discovered his true calling in battle. He's known for his bravery and unwavering loyalty to his friends."

}

```


In [ ]:
# 원-샷 학습: 출력 구조에 대한 예시를 제공합니다.
one_shot_template = """RPG 게임의 짧은 캐릭터 프로필을 만드세요. 이 형식만 사용하세요.:

{
  "description": "간단한 설명",
  "name": "캐릭터의 이름",
  "armor": "한 조각의 갑옷",
  "weapon": "하나 이상의 무기"
}
"""
one_shot_prompt = [
    {"role": "user", "content": one_shot_template}
]

# 출력을 생성합니다.
outputs = pipe(one_shot_prompt)
print(outputs[0]["generated_text"])

 ```json

{

  "description": "작은 저녁 잔치 속삭이 되어 있는 저녁 놀이에 참여하는 캐릭터",

  "name": "작은 저녁 잔치 속삭이",

  "armor": "작은 저녁 잔치 속삭이 착용한 옷",

  "weapon": "작은 저녁 잔치 속삭이 착용한 치마"

}

```


### **문법: 제약 샘플링** (개인 공부)

- 퓨샷(few-shot) 학습의 단점
    - 특정 출력이 생성되는 것을 명시적으로 막을 수 없다.
    - 모델에게 가이드와 지시 사항을 제공하지만 모델이 이를 완전히 따르지 않을 수 있다.
- **생성 모델의 출력을 제어하고 검증하기 위한 패키지**
    - Guidance : https://github.com/guidance-ai/guidance
    - Guardrails : https://github.com/guardrails-ai/guardrails
    - LMQL : https://github.com/eth-sri/lmql



---


#### **llama-cpp-python**
- llama-cpp-python은 llama.cpp 라이브러리의 파이썬 바인딩이다
    - 기본으로 설치되는 CPU 버전은 오랜 시간이 거리기때문에 CUDA같은 하드웨어 가속기를 지원한는 whl 파일을 깃허브에서 다우로드하여 설치한다.
    - https://github.com/abetlen/llama-cpp-python/releases
- 코랩에서 llama-cpp-python 설치하기
- https://github.com/abetlen/llama-cpp-python


In [ ]:
%%capture
# 사용하는 파이썬과 CUDA 버전에 맞는 llama-cpp-python 패키지를 설치하세요.
!pip install https://github.com/abetlen/llama-cpp-python/releases/download/v0.3.16-cu124/llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl



---



위 whl 파일 설치시 오류가 발생하면 아래 1단계 > 2단계 > 3단계 방법을 사용해 본다.

- **1단계: GPU 런타임 확인 및 설정**

In [ ]:
# GPU 사용 가능 여부 확인
import subprocess
import sys

print("🔍 현재 Python 버전 확인:")
print(f"Python {sys.version}\n")

print("🔍 GPU 사용 가능 여부 확인:")
try:
    gpu_info = subprocess.check_output(['nvidia-smi'], stderr=subprocess.STDOUT)
    print(gpu_info.decode('utf-8'))
    print("✅ GPU를 사용할 수 있습니다!")
    USE_GPU = True
except:
    print("❌ GPU를 사용할 수 없습니다. (CPU 모드로 진행)")
    print("💡 런타임 > 런타임 유형 변경 > T4 GPU 선택을 권장합니다.")
    USE_GPU = False

- **2단계: Colab 최적화 설치** (GPU 지원)

In [ ]:
# Colab에서 GPU 가속 llama-cpp-python 설치
# 이 방법이 가장 빠르고 안정적입니다

if USE_GPU:
    print("🚀 GPU 가속 버전 설치 중...")
    # CUDA 지원 버전 설치
    !CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python --force-reinstall --no-cache-dir -q
    print("✅ GPU 가속 버전 설치 완료!")
else:
    print("🐢 CPU 전용 버전 설치 중...")
    !pip install llama-cpp-python -q
    print("✅ CPU 버전 설치 완료!")

# 추가 필요 패키지 설치
!pip install huggingface-hub -q

print("\n📦 모든 패키지 설치 완료!")

- 3단계: 설치 검증

In [ ]:
# 설치 확인 및 버전 체크
try:
    from llama_cpp import Llama
    import llama_cpp

    print("✅ llama-cpp-python 설치 성공!")
    print(f"📌 버전: {llama_cpp.__version__}")

    # GPU 사용 가능 여부 확인
    if USE_GPU:
        print("🎮 GPU 가속이 활성화되었습니다.")
    else:
        print("💻 CPU 모드로 실행됩니다.")

except ImportError as e:
    print(f"❌ 설치 실패: {e}")
    print("다시 설치를 시도해주세요.")



---



In [ ]:
import gc
import torch
del model, tokenizer, pipe

# 메모리를 비웁니다.
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from llama_cpp.llama import Llama

# Phi-3를 로드합니다.
llm = Llama.from_pretrained(
    repo_id="microsoft/Phi-3-mini-4k-instruct-gguf",
    filename="*fp16.gguf",
    n_gpu_layers=-1,
    n_ctx=4096,
    verbose=False
)

# fp16.gguf

./Phi-3-mini-4k-instruct-fp16.gguf:   0%|          | 0.00/7.64G [00:00<?, ?B/s]

In [ ]:
# 출력을 생성합니다.
output = llm.create_chat_completion(
    messages=[
        {"role": "user", "content": "JSON 형식으로 RPG용 전사를 만듭니다."},
    ],
    response_format={"type": "json_object"},
    temperature=0,
)['choices'][0]['message']["content"]

In [ ]:
import json

# JSON 문자열을 로드합니다.
json_output = json.dumps(json.loads(output), indent=4)
print(json_output)

##프롬프트 기법 참고 기사 (AI에 다양한 답변을 요구하는 방법)
https://www.aitimes.com/news/articleView.html?idxno=203264
- 버벌라이즈드 샘플링

## 클로드 프롬프트 엔지니어링
https://docs.claude.com/ko/docs/build-with-claude/prompt-engineering/overview

# [미션] 프롬프트 엔지니어링 적용하기
1. [필수] 적절한 모델 선정하여(2개 이상 선정 후 비교) **고급 프롬프트 엔지니어링 방법** 따라서 적용해보기.
  - 복잡한 프롬프트
  - 문맥 내 학습: 예시 제공 방법
  - 프롬프트 체인: 문서 쪼개기
  - CoT 추론
  - ToT: 중간 단계 탐색
  - 출력검증: 예시 제공
  - 파일명: 20251021_고급프롬프트 엔지니어링 방법_장지현.ipynb

2. [선택] base model 선정 -> 미세튜닝(+ 추가 데이터 추가학습) -> 성능 지표 검증 -> 프롬프트 엔지니어링 적용 -> 출력 검증
  - 목표 정의 ( ~ 하는 OOO LLM 모델 만들기 )
  & 성능 지표 선택
  - LLM 모델링
    - 베이스모델 선정
    - 미세 튜닝(+ 추가 데이터 학습)
    - 성능 지표 검증
    - 프롬프트 엔지니어링 적용
    - 출력 검증
    - 파일명: 20251021_OOO LLM 모델 만들기_장지현.ipynb

## 1. 미션 1 - 고급 프롬프트 엔지니어링 방법 적용

(2개 이상 모델 선정 후 비교)

###  모델 선정
1. `Qwen/Qwen2.5-3B-Instruct` (소형)
  - 한글 성능 우수 (다국어 모델)
  - 빠른 추론 속도
  - 작은 파라미터 (3B)





In [ ]:
MODELS = {
    "Qwen2.5-3B": {
        "id": "Qwen/Qwen2.5-3B-Instruct",
        "size": "3B",
        "description": "소형, 빠른 추론"
    },
    "Phi-3-mini": { # 교안에서 씀
        "id": "microsoft/Phi-3-mini-4k-instruct",
        "size": "3.8B",
        "description": "중소형, MS 모델, 교안 메인"
    }
}

### 모델별 최적 do_sample, 추가 파라미터 설정 가이드

| 모델 | do_sample | 이유 | 추가 파라미터 |
|------|-----------|------|---------------|
| **Llama 2/3** | ✅ True | • 대화형 모델로 창의성/다양성 중요<br>• 긴 텍스트 생성에 적합<br>• 반복 생성 방지 | `temperature=0.7`<br>`top_p=0.9` |
| **GPT-2** | ✅ True | • 창작 글쓰기에 특화<br>• 다양한 스타일 생성 필요<br>• 작은 모델이라 샘플링으로 품질 향상 | `temperature=0.8`<br>`top_k=50` |
| **Phi-3** | ❌ False | • Instruction-following 특화<br>• 정확한 답변이 우선<br>• 작은 모델로 일관성 중요 | `max_new_tokens=512` |
| **CodeLlama** | ❌ False | • 코드 정확성이 필수<br>• 문법 오류 최소화<br>• 실행 가능한 코드 생성 목표 | `num_beams=5`<br>`early_stopping=True` |
| **Mistral 7B** | ✅ True | • 범용 대화 모델<br>• 균형잡힌 창의성과 정확성<br>• 효율적인 7B 모델 | `temperature=0.6`<br>`top_p=0.95` |
| **Falcon** | ✅ True | • 대규모 다목적 모델<br>• 창의적 작업에 강점<br>• 다양한 도메인 지원 | `temperature=0.7`<br>`repetition_penalty=1.1` |
| **T5/Flan-T5** | ❌ False | • Task-specific fine-tuning<br>• 정확한 변환 작업 (번역, 요약)<br>• Encoder-Decoder 구조 특성 | `num_beams=4`<br>`length_penalty=2.0` |
| **BERT-based** | ❌ False | • 분류/이해 작업 중심<br>• 마스크 언어 모델<br>• 생성보다는 이해에 특화 | 생성 작업에 비추천 |
| **ChatGLM** | ✅ True | • 중국어 대화 특화<br>• 대화형 상호작용 설계<br>• 문화적 뉘앙스 표현 필요 | `temperature=0.8`<br>`top_p=0.8` |
| **Vicuna** | ✅ True | • ChatGPT 스타일 모방<br>• 자연스러운 대화 중요<br>• 다양한 톤과 스타일 지원 | `temperature=0.7`<br>`top_p=0.9` |

## 📋 사용 목적별 가이드

### 🎨 **창의적 작업** → do_sample=True
- **모델**: Llama, GPT-2, Mistral
- **용도**: 소설 쓰기, 브레인스토밍, 대화

### 🎯 **정확성 중요** → do_sample=False  
- **모델**: Phi-3, CodeLlama, T5
- **용도**: 번역, 요약, 코딩, QA

### ⚖️ **균형 잡힌 접근**
- **High creativity**: `temperature=0.8-1.0`
- **Moderate creativity**: `temperature=0.6-0.7`  
- **Conservative**: `temperature=0.1-0.3`

## 💡 실전 팁

1. **대화형 챗봇**: 항상 `do_sample=True`
2. **코드 생성**: 항상 `do_sample=False`
3. **번역**: `do_sample=False` + beam search
4. **창작**: `do_sample=True` + 높은 temperature

In [ ]:
# 공통 설정
import gc
import torch
import warnings
import logging
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from collections import Counter
import re
import json
import time

# 경고 숨기기
warnings.filterwarnings("ignore")
logging.getLogger("transformers").setLevel(logging.ERROR)



# 유틸리티
def load_model(model_id):
    """모델 로드"""
    print(f"\n⏳ 모델 로딩 중: {model_id}")

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="cuda",
        torch_dtype="auto",
        trust_remote_code=True,
        attn_implementation="eager"
    )

    tokenizer = AutoTokenizer.from_pretrained(model_id)

    # pad_token 설정
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        return_full_text=False,
        max_new_tokens=500,
        do_sample=True,
        temperature=0.5,
        top_p=0.9,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id
    )

    print("✅ 로딩 완료")
    return model, tokenizer, pipe


def cleanup_model(*objs):
    # objs: model, tokenizer, pipe 등 전달
    for o in objs:
        try:
            del o
        except Exception:
            pass
    gc.collect()
    torch.cuda.empty_cache()
    print("👌 메모리 정리 완료")


def measure_time(func):
    """실행 시간 측정 데코레이터"""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - start
        return result, elapsed
    return wrapper


def print_comparison(results):
    """결과 비교 출력"""
    print("\n" + "="*70)
    print("결과 비교")
    print("="*70)

    for model_name in MODELS.keys():
        print(f"\n[{model_name}]")
        print("-"*70)
        print(f"출력: {results[model_name]['output']}")
        print(f"실행 시간: {results[model_name]['time']:.2f}초")

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# 아주 간단한 프롬프트로 테스트
simple_prompt = "안녕하세요. 인공지능에 대해 한 줄로 설명해주세요."

for model_name, model_info in MODELS.items():
    model, tokenizer, pipe = load_model(model_info["id"])

    messages = [{"role": "user", "content": simple_prompt}]
    output = pipe(messages)[0]["generated_text"]

    print(f"{model_name}: {output}")
    cleanup_model(model, tokenizer, pipe)


⏳ 모델 로딩 중: Qwen/Qwen2.5-3B-Instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 로딩 완료
Qwen2.5-3B: 안녕하세요. 인공지능은 컴퓨터 시스템이 지정된 작업을 이해하고 수행하는 능력입니다.
👌 메모리 정리 완료

⏳ 모델 로딩 중: microsoft/Phi-3-mini-4k-instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 로딩 완료
Phi-3-mini:  인공지능은 기계학습을 통해 사람들이 개발하는 컴퓨터 시스템의 일종입니다. 이는 다양한 기법을 활용하여 다음과 같은 종류의 기능을 수행합니다:

- 문자 추출 (OCR) : 영상에서 판단하여 문자를 읽어내고, 그 문장을 입력 데이터로 만들어 실행할 수 있습니다.
- 질문 및 답변 : 사용자가 질문을 제출하면, 인공지능이 정확한 답변을 제공합니다.
- 생산 및 운영 업무 : 컴퓨터 시스템이 작업을 수행하거나 사회적 활동을 조율할 수 있습니다.
- 데이터 분석 : 많은 데이터를 분석하고 올바른 결론을 도출하여 사회적 활동을 수행합니다.
- 웹 서비스 : 사용자들이 웹 페이지에서 인공지능을 통해 서비스를 받을 수 있습니다.
👌 메모리 정리 완료


### 복잡한 프롬프트


In [ ]:

# 복잡한 프롬프트 구성
text = """
인공지능(AI)은 단순한 계산 자동화 기술을 넘어, 인간의 사고방식을
모방하고 스스로 학습하는 시스템으로 진화하고 있다. 특히 딥러닝 기반의
언어 모델과 이미지 생성 모델은 인간의 창의성을 위협할 정도의 표현력을
보이고 있으며, 의료, 금융, 교육 등 다양한 분야에서 인간의 판단을 보조하거나
대체하는 수준에 도달했다.

하지만 이러한 기술의 발전은 동시에 여러 윤리적 문제를 동반하고 있다.
대표적으로 ‘편향된 학습 데이터’로 인한 차별, 자동화로 인한 일자리 감소,
그리고 인공지능의 의사결정 과정이 불투명하다는 점 등이 있다.
AI가 내린 결정을 사람조차 설명할 수 없는 ‘블랙박스 문제’는 사회적 신뢰를
저하시킬 수 있는 중요한 요인이다.

최근에는 AI가 생성한 이미지나 텍스트가 실제 인간의 창작물과 구별하기
어려운 수준으로 발전하면서, ‘저작권’과 ‘창작자성’에 대한 논의도 활발하다.
특히 생성형 AI가 인터넷상의 데이터를 무단으로 학습하는 과정에서
원저작자의 권리가 침해되는 사례가 보고되고 있다.

한편, 기술 발전을 멈출 수 없다는 현실 속에서, ‘책임 있는 AI 개발’이라는
새로운 방향성이 제시되고 있다. AI 모델의 학습 과정에서 데이터 출처를
투명하게 공개하고, 편향을 줄이는 정량적 기준을 마련하는 것이
그 핵심 과제로 떠오르고 있다. 또한 AI가 사회적으로 미칠 영향을
사전에 예측하고, 인류 전체의 이익을 고려하는 거버넌스 체계 구축이
중요하다는 목소리도 커지고 있다.

결국 AI의 발전은 기술만의 문제가 아니라, 인간이 기술을 어떻게
이해하고 활용할 것인가의 문제로 귀결된다. 기술의 속도보다
윤리적 판단의 속도가 늦어질 때, 사회는 그 대가를 치르게 될지도 모른다.
"""

persona = "당신은 AI 전문 기술 작가입니다.\n"
instruction = "제공된 텍스트의 핵심 내용을 요약하세요.반드시 한국어로만 답변하세요.\n"
context = "이 요약은 비전공자도 이해할 수 있어야 합니다.\n"
data_format = "3줄 이내로 요약하고, 핵심 키워드를 포함하세요.\n"
audience = "대상 독자는 AI에 관심 있는 일반인입니다.\n"
tone = "쉽고 친근한 어조를 사용하세요.\n"
data = f"요약할 텍스트: {text}"

complex_prompt = persona + instruction + context + data_format + audience + tone + data

results_complex = {}

for model_name, model_info in MODELS.items():
    model, tokenizer, pipe = load_model(model_info["id"])

    messages = [{"role": "user", "content": complex_prompt}]

    @measure_time
    def generate():
        return pipe(messages)[0]["generated_text"]

    output, elapsed = generate()
    results_complex[model_name] = {"output": output, "time": elapsed}

    cleanup_model(model, tokenizer, pipe)

print_comparison(results_complex)


⏳ 모델 로딩 중: Qwen/Qwen2.5-3B-Instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 로딩 완료
👌 메모리 정리 완료

⏳ 모델 로딩 중: microsoft/Phi-3-mini-4k-instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 로딩 완료
👌 메모리 정리 완료

결과 비교

[Qwen2.5-3B]
----------------------------------------------------------------------
출력: 인공지능은 이제 인간의 창의성까지 위협할 수 있는 강력한 기술이 되었습니다. 하지만 이 발전은 여러 윤리적 문제를 야기합니다. 편향된 학습 데이터로 인한 차별부터 저작권 논란까지, AI의 결정이 명확하지 않아 사회 신뢰를 잃을 수도 있습니다. 따라서 '책임있는 AI 개발'이 필요하며, 데이터 공개와 편향 해소를 위한 기준이 마련되어야 합니다. 결국 AI는 우리 모두가 어떻게 이를 이해하고 활용하느냐에 달려있습니다.
실행 시간: 7.00초

[Phi-3-mini]
----------------------------------------------------------------------
출력:  AI 기술은 잘 발전하며, 인간의 창의성을 포유되지만, 이에 함께 있는 윤리적 문제가 있으므로 활동은 점점 잠재적 불안정성으로 이어질 수 있다. '블랙박스 문제', 인간의 창작 권리, 인터넷 데이터 획득 관리, 저작권 문제, 사회적 영향 예측, 인류 이익 추구 계획이 중요한 요소들이 있음. 이러한 문제를 해결하려면 AI 개발 과정에서 투명한 기술 사용, 평가 방법 구축, 인류 이익 측정 방법 개발이 중요하다.
실행 시간: 16.52초


### 문맥 내 학습: 예시 제공

In [ ]:
# Zero-shot
zeroshot_prompt = [
    {"role": "user", "content": "다음 리뷰의 감정을 분석하세요: '음식이 정말 맛있었어요!'"}
]

# One-shot
oneshot_prompt = [
    {"role": "user", "content": "리뷰: '서비스가 별로였어요.'"},
    {"role": "assistant", "content": "감정: 부정"},
    {"role": "user", "content": "리뷰: '음식이 정말 맛있었어요!'"}
]

# Few-shot
fewshot_prompt = [
    {"role": "user", "content": "리뷰: '서비스가 별로였어요.'"},
    {"role": "assistant", "content": "감정: 부정"},
    {"role": "user", "content": "리뷰: '분위기가 좋네요!'"},
    {"role": "assistant", "content": "감정: 긍정"},
    {"role": "user", "content": "리뷰: '그냥 그래요.'"},
    {"role": "assistant", "content": "감정: 중립"},
    {"role": "user", "content": "리뷰: '음식이 정말 맛있었어요!'"}
]

shot_types = {
    "Zero-shot": zeroshot_prompt,
    "One-shot": oneshot_prompt,
    "Few-shot": fewshot_prompt
}

results_shots = {model_name: {} for model_name in MODELS.keys()}

for model_name, model_info in MODELS.items():
    model, tokenizer, pipe = load_model(model_info["id"])

    for shot_name, prompt in shot_types.items():
        output = pipe(prompt)[0]["generated_text"]
        results_shots[model_name][shot_name] = output

    cleanup_model(model, tokenizer, pipe)

# 비교 출력
print("\nShot 방식별 비교:")
for shot_name in shot_types.keys():
    print(f"\n[{shot_name}]")
    print("-"*70)
    for model_name in MODELS.keys():
        print(f"{model_name}: {results_shots[model_name][shot_name]}")


⏳ 모델 로딩 중: Qwen/Qwen2.5-3B-Instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 로딩 완료
👌 메모리 정리 완료

⏳ 모델 로딩 중: microsoft/Phi-3-mini-4k-instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 로딩 완료
👌 메모리 정리 완료

Shot 방식별 비교:

[Zero-shot]
----------------------------------------------------------------------
Qwen2.5-3B: 이 리뷰는 매우 긍정적인 감정을 나타내고 있습니다. 사용자는 음식이 "맛있었다"고 표현했으며, 이는 명확한 만족감과 좋은 경험을 의미합니다. 단어 "실로"는 강조를 더해주며 더욱 높은 평가를 전달하고 있습니다.
Phi-3-mini:  이 리뷰에서 나타나는 감정은 환호적인 상태를 나타내고 있습니다. 주어진 문장 "음식이 정말 맛있었어요!"는 음식이 맛있다고 얘기하고 있으며, 이는 음식을 선호하거나 기쁩니다. 이 결과로 나오는 감정은 환호적인 반면에 조용한 정직도가 보일 수 있습니다. 이 감정을 분석하려면 음식이 맛있다는 것을 인지하고 그 결과로 인해 사람의 피부나 몸에 좋은 영향을 미치는 것을 알 수 있습니다. 이 리뷰는 음식 품질의 좋은 성질을 강조하고 있으며, 이는 음식을 즐기는 경험을 표현하며 환호적인 감정을 나타낼 수 있습니다.

[One-shot]
----------------------------------------------------------------------
Qwen2.5-3B: 감정: 긍정
Phi-3-mini:  감정: 기쁨

분석 내용:

- 특정 서비스에 대해 주장하고 있으며, 이는 음식 제품에 대한 긍정적인 평가를 의미합니다.

- 광범위한 정보를 제공하지 않으므로 상세한 내용을 분석할 수 없습니다. 그러나 일반적으로 음식이 맛있다는 말은 음식 제품이 원시적인 즐거 umami 등의 특성을 잃지 않고 완전히 맛보여주고 있다는 것을 의미합니다.

- 감정: 기쁨 - 음식이 맛있다는 말에 따라, 음식 제품이 소중한 경험을 주는 것으로 감정적으로 감사드립니다.

[Few-shot]
----------------------------------------------------------------------
Qw

### 프롬프트 체인: 문서 쪼개기
- 이름에 '민주'가 들어가니 민주주의 설명이 나온다 ,,

In [ ]:
results_chain = {}

for model_name, model_info in MODELS.items():
    model, tokenizer, pipe = load_model(model_info["id"])

    # 1단계: 이름 생성
    step1_prompt = [
        {"role": "user", "content": "한국 여자 아이 이름 한 개를 지어줘. 이름만 답해줘."}
    ]
    name = pipe(step1_prompt, max_new_tokens=50)[0]["generated_text"].strip()

    # 2단계: 캐릭터 설정
    step2_prompt = [
        {"role": "user", "content": f"'{name}'라는 이름의 판타지 소설 주인공 설정을 2-3줄로 만들어줘."}
    ]
    character = pipe(step2_prompt, max_new_tokens=200)[0]["generated_text"].strip()

    # 3단계: 홍보 문구
    step3_prompt = [
        {"role": "user", "content": f"다음 캐릭터에 대한 흥미로운 홍보 문구를 작성해줘:\n\n{character}"}
    ]
    promo = pipe(step3_prompt, max_new_tokens=200)[0]["generated_text"].strip()

    results_chain[model_name] = {
        "step1_name": name,
        "step2_character": character,
        "step3_promo": promo
    }

    cleanup_model(model, tokenizer, pipe)

# 비교 출력
print("\n프롬프트 체인 결과:")
for model_name, chain in results_chain.items():
    print(f"\n[{model_name}]")
    print("-"*70)
    print(f"1단계 (이름): {chain['step1_name']}")
    print(f"2단계 (캐릭터): {chain['step2_character']}")
    print(f"3단계 (홍보): {chain['step3_promo']}")


⏳ 모델 로딩 중: Qwen/Qwen2.5-3B-Instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 로딩 완료
👌 메모리 정리 완료

⏳ 모델 로딩 중: microsoft/Phi-3-mini-4k-instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 로딩 완료
👌 메모리 정리 완료

프롬프트 체인 결과:

[Qwen2.5-3B]
----------------------------------------------------------------------
1단계 (이름): 서연
2단계 (캐릭터): '서연', 그림 같은 투명한 피부와 검은 눈동자에 빛나는 파란 머리카락이 돋보이는 소녀. 서연은 세상에서 가장 순수하고 따뜻한 존재로, 언제나 다른 사람들을 위해 행동하는 천사처럼 아름다운 그녀는 판타지 세계를 더욱 밝게 비추는 빛이다.
3단계 (홍보): "서연: 천사의 미소, 순수함의 탄생"
이 광고 문구는 서연이라는 캐릭터가 가지고 있는 특징을 잘 전달합니다. '천사의 미소'는 그녀의 따뜻함과 사랑스러움을 표현하며, '순수함의 탄생'은 그녀가 세상에 가져다주는 영향력을 강조합니다. 또한, "판타지 세계를 더욱 밝게 비추는 빛"이라는 표현은 서연이 주는 긍정적인 에너지를 상징적으로 보여줍니다.

[Phi-3-mini]
----------------------------------------------------------------------
1단계 (이름): 민주


(Note: The provided solution is a common Korean name, Minju, which means "democracy". It's chosen as an example of simplicity and directness in response to the instruction.)
2단계 (캐릭터): 민주는 결정적인 명사로서 민주주의를 나타내는 명사입니다. 한국에서 일본 양화 문제에 대한 개방적인 평화와 자유를 가진 민주주의를 바탕으로 이러한 이름을 선택하였습니다. 민주주의의 중요성과 역사적 영향을 담고 있는 이름은 현대 문화에서도 그리스어 영문에서도 민주주의를 의미하는 단어인 'D
3단계 (홍보): "민주주의는 우리 모두에게 헌신과 불안거나 불만을 잔들지 않는 데 중요한 원칙이 있습니다."


"민주주의는 국민의 

### CoT 추론

- 결과: zeroshot 보다 one shot이 더 효과적



In [ ]:
# Zero-shot CoT
zeroshot_cot_prompt = [
    {"role": "user", "content": """
식당에는 사과 23개가 있었습니다.
20개를 점심 만드는 데 사용하고 6개를 더 샀다면,
남은 사과는 몇 개일까요?

계산 과정 단계별로 생각해 봅시다.
"""}
]

# Few-shot CoT
fewshot_cot_prompt = [
    {"role": "user", "content": "로저는 테니스공 5개를 가지고 있습니다. 캔 2개를 더 샀고, 각 캔에는 공이 3개씩 들어있습니다. 이제 몇 개일까요?"},
    {"role": "assistant", "content": "로저는 공 5개로 시작했습니다. 캔 2개 × 공 3개 = 6개입니다. 5 + 6 = 11개입니다. 답은 11개입니다."},
    {"role": "user", "content": "식당에는 사과 23개가 있었습니다. 20개를 점심 만드는 데 사용하고 6개를 더 샀다면, 남은 사과는 몇 개일까요?"}
]

cot_types = {
    "Zero-shot CoT": zeroshot_cot_prompt,
    "Few-shot CoT": fewshot_cot_prompt
}

results_cot = {model_name: {} for model_name in MODELS.keys()}

for model_name, model_info in MODELS.items():
    model, tokenizer, pipe = load_model(model_info["id"])

    for cot_name, prompt in cot_types.items():
        output = pipe(prompt)[0]["generated_text"]
        results_cot[model_name][cot_name] = output

    cleanup_model(model, tokenizer, pipe)

# 비교 출력
print("\nCoT 방식별 비교:")
for cot_name in cot_types.keys():
    print(f"\n[{cot_name}]")
    print("-"*70)
    for model_name in MODELS.keys():
        print(f"\n{model_name}:")
        print(results_cot[model_name][cot_name][:200] + "...")


⏳ 모델 로딩 중: Qwen/Qwen2.5-3B-Instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 로딩 완료
👌 메모리 정리 완료

⏳ 모델 로딩 중: microsoft/Phi-3-mini-4k-instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 로딩 완료
👌 메모리 정리 완료

CoT 방식별 비교:

[Zero-shot CoT]
----------------------------------------------------------------------

Qwen2.5-3B:
물론입니다! 문제를 단계별로 해결해 보겠습니다.

1) **초기 상태**: 식당에 있는 사과의 수는 23개입니다.
   
2) **사과 사용**: 20개를 점심을 만들 때 사용합니다.
   - 남아있는 사과의 수 = 초기 상태의 사과 수 - 사용한 사과 수
   - 남아있는 사과의 수 = 23 - 20 = 3개
   
3) **추가 구입**: 남아있는 3개...

Phi-3-mini:
 1. 식당에서 처음 들어왔을 때 사과의 수: 23개

2. 점심 만들기 위해 사용했던 사과의 수: 20개

3. 새로운 사과 추가: 6개

4. 총 사과의 수 계산: 23개 (첫 인덱스) + 6개 (두 번째 인덱스) = 29개

5. 사과가 사용된 사과 수: 20개

6. 남은 사과의 수 계산: 29개 (총 사과 수) - 20개 (사용된 사과 수) = 9...

[Few-shot CoT]
----------------------------------------------------------------------

Qwen2.5-3B:
처음으로 식당에 있는 사과의 수는 23개였습니다. 점심을 위해 20개를 사용했으므로 남은 사과는 23 - 20 = 3개가 됩니다.

그 다음에 6개를 추가로 샀기 때문에, 현재 남아있는 사과의 수는 3 + 6 = 9개가 됩니다.

따라서, 지금 남아있는 사과의 수는 9개입니다....

Phi-3-mini:
 식당에서 촉구한 사과의 수는 23개였습니다. 점심 만들기에 사용할 사과는 20개입니다. 남은 사과는 23 - 20 = 3개입니다. 그런데 6개를 더 샀으므로, 3 + 6 = 9개입니다. 답은 9개입니다....


### 자기 일관성 (Self-Consistency)

In [ ]:
# 답변 추출 함수
def extract_answer(text):
    """응답에서 숫자 추출"""
    numbers = re.findall(r'\d+', text)
    # 문제의 숫자 제외
    filtered = [int(n) for n in numbers if int(n) not in [23, 20, 6]]
    return filtered[-1] if filtered else None

question = """
식당에는 사과 23개가 있었습니다.
20개를 점심 만드는 데 사용하고 6개를 더 샀다면,
남은 사과는 몇 개일까요?
"""

self_consistency_prompt = [
    {"role": "user", "content": question}
]

results_consistency = {}

for model_name, model_info in MODELS.items():
    model, tokenizer, pipe = load_model(model_info["id"])

    answers = []
    n_samples = 7

    print(f"\n[{model_name}] {n_samples}번 샘플링 중...")

    for i in range(n_samples):
        output = pipe(
            self_consistency_prompt,
            do_sample=True,
            temperature=0.8,
            max_new_tokens=300
        )[0]["generated_text"]

        answer = extract_answer(output)
        if answer:
            answers.append(answer)
            print(f"  시도 {i+1}: {answer}개")

    # 다수결
    if answers:
        vote_counts = Counter(answers)
        final_answer, count = vote_counts.most_common(1)[0]
        confidence = count / n_samples

        results_consistency[model_name] = {
            "answers": answers,
            "final": final_answer,
            "confidence": confidence,
            "votes": dict(vote_counts)
        }

    cleanup_model(model, tokenizer, pipe)

# 비교 출력
print("\n자기 일관성 결과:")
for model_name, result in results_consistency.items():
    print(f"\n[{model_name}]")
    print("-"*70)
    print(f"수집된 답변: {result['answers']}")
    print(f"최종 답변: {result['final']}개")
    print(f"신뢰도: {result['confidence']:.1%}")
    print(f"득표 분포: {result['votes']}")


⏳ 모델 로딩 중: Qwen/Qwen2.5-3B-Instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 로딩 완료

[Qwen2.5-3B] 7번 샘플링 중...
  시도 1: 9개
  시도 2: 9개
  시도 3: 9개
  시도 4: 9개
  시도 5: 9개
  시도 6: 9개
  시도 7: 3개
👌 메모리 정리 완료

⏳ 모델 로딩 중: microsoft/Phi-3-mini-4k-instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 로딩 완료

[Phi-3-mini] 7번 샘플링 중...
  시도 1: 9개
  시도 2: 16개
  시도 3: 26개
  시도 4: 29개
  시도 5: 26개
  시도 6: 9개
  시도 7: 3개
👌 메모리 정리 완료

자기 일관성 결과:

[Qwen2.5-3B]
----------------------------------------------------------------------
수집된 답변: [9, 9, 9, 9, 9, 9, 3]
최종 답변: 9개
신뢰도: 85.7%
득표 분포: {9: 6, 3: 1}

[Phi-3-mini]
----------------------------------------------------------------------
수집된 답변: [9, 16, 26, 29, 26, 9, 3]
최종 답변: 9개
신뢰도: 28.6%
득표 분포: {9: 2, 16: 1, 26: 2, 29: 1, 3: 1}


### ToT: 중간 단계 탐색

In [ ]:
tot_prompt = [
    {"role": "user", "content": """
세 명의 전문가가 이 질문에 답한다고 가정해 보세요.
모든 전문가는 자신의 생각의 한 단계를 적어 그룹원들과 공유합니다.
그런 다음 모든 전문가가 다음 단계로 넘어가는 식으로 진행합니다.
어느 시점에서든 자신이 틀렸다는 것을 깨닫는 전문가가 있으면 그 자리에서 나갑니다.

질문: "식당에는 사과 23개가 있었습니다. 20개를 점심으로 만들고 6개를 더 샀다면,
사과는 몇 개일까요?"

각 전문가의 추론 과정을 보여주고, 최종 답을 도출하세요.
"""}
]

results_tot = {}

for model_name, model_info in MODELS.items():
    model, tokenizer, pipe = load_model(model_info["id"])

    output = pipe(tot_prompt, max_new_tokens=800)[0]["generated_text"]
    results_tot[model_name] = output

    cleanup_model(model, tokenizer, pipe)

# 비교 출력
print("\nToT 결과:")
for model_name, output in results_tot.items():
    print(f"\n[{model_name}]")
    print("-"*70)
    print(output[:400] + "...")


⏳ 모델 로딩 중: Qwen/Qwen2.5-3B-Instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 로딩 완료
👌 메모리 정리 완료

⏳ 모델 로딩 중: microsoft/Phi-3-mini-4k-instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 로딩 완료
👌 메모리 정리 완료

ToT 결과:

[Qwen2.5-3B]
----------------------------------------------------------------------
네, 각 전문가가 문제를 해결하는 과정을 따라보도록 하겠습니다.

### 전문가 A

**단계 1:** 초기 상태에서 사과는 23개입니다.
**단계 2:** 20개를 점심으로 사용했으므로 남은 수량은 23 - 20 = 3개입니다.
**단계 3:** 그리고 6개를 추가적으로 샀으므로, 현재의 사과의 총 수는 3 + 6 = 9개입니다.

**결론:** 사과는 9개입니다.

---

### 전문가 B

**단계 1:** 초기 상태에서 사과는 23개입니다.
**단계 2:** 20개를 점심으로 사용했으므로 남은 수량은 23 - 20 = 3개입니다.
**단계 3:** 그리고 6개를 추가적으로 샀으므로, 현재의 사과의 총 수는 3 + 6 = 9개입니다.

**결论（注意：原文中“結論”应为“结论”）:** 사과는 9개입니다....

[Phi-3-mini]
----------------------------------------------------------------------
 전문가1: 먼저 20개를 점심으로 만들었으므로, 남은 사과는 23 - 20 = 3개가 남았습니다. 그런 다음 6개를 더 준 것이므로, 3 + 6 = 9개가 됩니다. 따라서 당시 식당에 있던 사과의 수는 9개였습니다.

전문가2: 점심 먹기 위해 20개를 사용했으므로, 남은 사과는 23 - 20 = 3개가 남았습니다. 그런 다음 6개를 더 준 것이므로, 3 + 6 = 9개가 됩니다. 따라서 당시 식당에 있던 사과의 수는 9개였습니다.

전문가3: 점심 먹기 전에 23개의 사과가 있었습니다. 20개를 사용했으므로, 남은 사과는 23 - 20 = 3개가 남았습니다. 그런 다음 6개를 더 준 것이므로, 3 + 6 = 9개가 됩니다. 따라서 당시 식당에 있던 사과의 수는 9개였습니다.

따라서 모든 전문가는 

### 출력 검증: 예시 제공

In [ ]:
import re
import json

json_template = """
RPG 게임의 짧은 캐릭터 프로필을 만드세요.
반드시 이 JSON 형식만 사용하세요:

{
  "name": "캐릭터 이름",
  "class": "직업",
  "weapon": "무기",
  "skill": "특수 능력"
}

예시:
{
  "name": "아리아",
  "class": "마법사",
  "weapon": "지팡이",
  "skill": "화염구"
}

이제 전사 캐릭터를 만들어주세요.
"""

validation_prompt = [
    {"role": "user", "content": json_template}
]

def extract_and_parse_json(output):
    """개선된 JSON 추출 및 파싱"""

    # 1. 마크다운 코드 블록에서 추출
    patterns = [
        r'```(?:json)?\s*(\{.*?\})\s*```',  # ```json { ... } ```
        r'```\s*(\{.*?\})\s*```',           # ``` { ... } ```
        r'\{.*\}',                          # 직접 JSON 객체
    ]

    for pattern in patterns:
        match = re.search(pattern, output, re.DOTALL)
        if match:
            json_candidate = match.group(1) if match.groups() else match.group()

            try:
                parsed = json.loads(json_candidate.strip())
                return parsed, "✅ JSON 파싱 성공"
            except json.JSONDecodeError:
                continue

    return None, "❌ 유효한 JSON을 찾을 수 없음"

# 검증 코드 수정
results_validation = {}

for model_name, model_info in MODELS.items():
    model, tokenizer, pipe = load_model(model_info["id"])

    output = pipe(validation_prompt, max_new_tokens=200)[0]["generated_text"]

    # 개선된 JSON 파싱
    parsed, validation_msg = extract_and_parse_json(output)
    is_valid = parsed is not None

    if is_valid:
        # 필수 필드 검증
        required = ["name", "class", "weapon", "skill"]
        missing = [f for f in required if f not in parsed]
        if missing:
            validation_msg += f" (누락 필드: {missing})"

    results_validation[model_name] = {
        "output": output,
        "valid": is_valid,
        "parsed": parsed,
        "message": validation_msg
    }

    cleanup_model(model, tokenizer, pipe)

# 결과 출력
print("\n개선된 출력 검증 결과:")
for model_name, result in results_validation.items():
    print(f"\n[{model_name}]")
    print("-"*70)
    print(f"원본 출력:\n{result['output']}")
    print(f"\n검증: {result['message']}")
    if result['parsed']:
        print(f"파싱 데이터:\n{json.dumps(result['parsed'], indent=2, ensure_ascii=False)}")


⏳ 모델 로딩 중: Qwen/Qwen2.5-3B-Instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 로딩 완료
👌 메모리 정리 완료

⏳ 모델 로딩 중: microsoft/Phi-3-mini-4k-instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 로딩 완료
👌 메모리 정리 완료

개선된 출력 검증 결과:

[Qwen2.5-3B]
----------------------------------------------------------------------
원본 출력:
{
  "name": "브루스",
  "class": "전사",
  "weapon": "검",
  "skill": "대형 칼날 던질기"
}

검증: ✅ JSON 파싱 성공
파싱 데이터:
{
  "name": "브루스",
  "class": "전사",
  "weapon": "검",
  "skill": "대형 칼날 던질기"
}

[Phi-3-mini]
----------------------------------------------------------------------
원본 출력:
 ```json

{

  "name": "전사 민수",
  "class": "전사",
  "weapon": "짱구",
  "skill": "방어융합"

}

```

검증: ✅ JSON 파싱 성공
파싱 데이터:
{
  "name": "전사 민수",
  "class": "전사",
  "weapon": "짱구",
  "skill": "방어융합"
}


In [ ]:
print("최종 종합 비교")
print("="*70)

comparison_table = {
    "모델": list(MODELS.keys()),
    "크기": [MODELS[m]["size"] for m in MODELS.keys()],
    "복잡한 프롬프트 속도": [f"{results_complex[m]['time']:.2f}초" for m in MODELS.keys()],
    "자기일관성 신뢰도": [f"{results_consistency[m]['confidence']:.1%}" for m in MODELS.keys()],
    "JSON 검증": [results_validation[m]['message'] for m in MODELS.keys()],
}

# 표 형식 출력
print("\n")
for key, values in comparison_table.items():
    print(f"{key:20s}", end="")
    for v in values:
        print(f"{str(v):25s}", end="")
    print()

print("\n" + "="*70)
print("결론")
print("="*70)

# 속도 비교
faster_model = min(MODELS.keys(), key=lambda m: results_complex[m]['time'])
print(f"속도: {faster_model} 우세")

# 신뢰도 비교
reliable_model = max(MODELS.keys(), key=lambda m: results_consistency[m]['confidence'])
print(f"일관성: {reliable_model} 우세")

print("\n각 모델의 특징:")
for model_name, info in MODELS.items():
    print(f"  • {model_name}: {info['description']}")

최종 종합 비교


모델                  Qwen2.5-3B               Phi-3-mini               
크기                  3B                       3.8B                     
복잡한 프롬프트 속도         7.00초                    16.52초                   
자기일관성 신뢰도           85.7%                    28.6%                    
JSON 검증             ✅ JSON 파싱 성공             ✅ JSON 파싱 성공             

결론
속도: Qwen2.5-3B 우세
일관성: Qwen2.5-3B 우세

각 모델의 특징:
  • Qwen2.5-3B: 소형, 빠른 추론
  • Phi-3-mini: 중소형, MS 모델, 교안 메인


## **모델 비교 분석 결과**

## 성능 비교

| 항목 | Qwen2.5-3B | Phi-3-mini |
|------|------------|------------|
| 크기 | 3B | 3.8B |
| 속도 | 7.00초 | 16.52초 |
| 일관성 | 85.7% | 28.6% |
| JSON 검증 | 성공 | 성공 |

---

## 주요 발견

### 속도
- Qwen이 2.36배 빠름
- 작은 모델이 더 빠른 역설

### 일관성
- Qwen: 7회 시도 중 6회 일치
- Phi-3: 7회 중 2회만 일치
- 신뢰도 차이 3배

### 구조화 출력
- 둘 다 JSON 생성 성공

---

## 원인 분석

### Qwen 우세 이유
- 다국어 토크나이저 (한국어 최적화)
- 효율적 아키텍처
- 안정적 학습 데이터

### Phi-3 부진 이유
- 영어 중심 설계
- 한국어 토큰 처리 비효율
- 출력 변동성 높음

---

## 테스트 한계

### 한국어 중심 평가
- Phi-3는 본래 영어에 강함
- 언어 차이로 인한 불리
- 영어 테스트 시 다른 결과 가능

---

## 결론

### Qwen2.5-3B 적합 상황
- 영어 외 다국어 서비스(한국어 포함)
- 빠른 응답 속도 필요
- 일관된 출력 필수
- 비용 효율 중시

### Phi-3-mini 적합 상황
- 영어 서비스
- MS 생태계
- 교육/연구

---

## 요약

한국어 태스크에서 Qwen2.5-3B가 속도와 일관성 모두 우세.
단, 영어 서비스에서는 Phi-3도 경쟁력 있을 수 있음.